In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import requests
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import os

sns.set_palette('Spectral')
sns.set_context('notebook', font_scale=1)
sns.set_style('whitegrid')
pd.set_option('display.max_columns', None)

import warnings

warnings.filterwarnings('ignore')

# **PREPARE FUNCTION**

In [2]:
# create function to transform timedelat to minute
def timedelta_to_hour(timedelta_str):
    # Konversi timedelta string ke timedelta object
    timedelta_obj = pd.to_timedelta(timedelta_str)
    
    # Hitung total detik dalam timedelta
    total_seconds = timedelta_obj.total_seconds()
    
    # Konversi total detik ke jam
    total_hour = total_seconds / 3600
    
    return total_hour

In [3]:
# create transform curency
def rupiah(value):
    str_value = str(value)
    separate_decimal = str_value.split(".")
    after_decimal = separate_decimal[0]
    before_decimal = separate_decimal[1]

    reverse = after_decimal[::-1]
    temp_reverse_value = ""

    for index, val in enumerate(reverse):
        if (index + 1) % 3 == 0 and index + 1 != len(reverse):
            temp_reverse_value = temp_reverse_value + val + "."
        else:
            temp_reverse_value = temp_reverse_value + val

    temp_result = temp_reverse_value[::-1]

    return "Rp " + temp_result + "," + before_decimal

In [4]:
def similairty_sentence(text1, text2):
    # Menggunakan regex untuk memisahkan teks berdasarkan simbol dan spasi
    tokens1 = ''.join(re.split(r'\W+', text1.lower()))
    tokens2 = ''.join(re.split(r'\W+', text2.lower()))

    # Menghitung frekuensi kata dalam masing-masing teks
    counter1 = Counter(tokens1)
    counter2 = Counter(tokens2)

    # Menghitung dot product
    dot_product = sum(counter1[word] * counter2[word] for word in counter1 if word in counter2)

    # Menghitung magnitudo dari vektor tiap teks
    magnitude1 = math.sqrt(sum(counter1[word] ** 2 for word in counter1))
    magnitude2 = math.sqrt(sum(counter2[word] ** 2 for word in counter2))

    # Menghindari pembagian oleh nol
    if magnitude1 == 0 or magnitude2 == 0:
        return 0

    # Menghitung cosine similarity
    similarity = dot_product / (magnitude1 * magnitude2)
    
    return similarity

In [5]:
def read_file(path, sheet=''):
    list_file = os.listdir(path)
    try:
        list_file.remove('.DS_Store')
    except:
        None
    
    msg = []
    for i in list_file:
        if sheet == '':
            msg.append(pd.read_excel(path + '/' + i))
        else:
            msg.append(pd.read_excel(path + '/' + i, sheet_name=sheet))
    msg = pd.concat(msg)
    msg.reset_index(drop=True, inplace=True)
    return msg

In [6]:
def get_work_date(x):
    x = str(x).split()[0]
    
    date_start = str(x).split()[0] + ' 07:00:00'
    date_end = str(x).split()[0] + ' 17:00:00'
    msg = {'start':date_start, 'end':date_end}
    return msg

In [7]:
def retext_equipment_name_t1(x):
    if x in ['SAKAI SAKAI SV 525 D']:
        msg = 'SAKAI SV'
    else:
        if x in ['KOMATSU PC200-8M1','KOMATSU PC200-10/S21','KOMATSU PC200-8M0','KOMATSU PC200-8']:
            msg = 'KOMATSU PC200'
        elif x in ['KOMATSU PC195LC', 'KOMATSU PC195LC','KOMATSU PC195-LC', 'KOMATSU PC195LC-8', 'KOMATSU PC195LC-8']:
            msg = 'KOMATSU PC195'
        elif x in ['HINO WU352R-HKMRJD8B','HINO WU352R-HKMRJD3L']:
            msg = 'HINO WU352R'
        elif x in ['KOBELCO SK 200','KOBELCO SK200-10','KOBELCO SK330-14','KOBELCO SK', 'KOBELCO SK 200']:
            msg = 'KOBELCO SK200'
        elif x in ['KOBELCO SK 330']:
            msg = 'KOBELCO SK330'
        elif x in ['KOMATSU PC300SE-8','KOMATSU PC300SE-8MO','KOMATSU PC300-8M0','KOMATSU PC300SE-8M0']:
            msg = 'KOMATSU PC300'
        elif x in ['KOMATSU PC400LCSE-8']:
            msg = 'KOMATSU PC400'
        elif x in ['KOMATSU D85ESS-2','KOMATSU D85E-SS-2']:
            msg = 'KOMATSU D85'
        elif x in ['KOMATSU D65P-12']:
            msg = 'KOMATSU D65'
        elif x in ['KOMATSU GD511A-1','KOMATSU GD535-5']:
            msg = 'KOMATSU GD'
        elif x in ['KOMATSU HM400-3R','KOMATSU HM400']:
            msg = 'KOMATSU HM400'
        elif x in ['KOMATSU PC500LC-10R']:
            msg = 'KOMATSU PC500'
        elif x in ['PAJERO SPORT','PAJERO']:
            msg = 'PAJERO'
        else:
            msg = x.split('-')[0]
    return msg

In [8]:
def retext_equipment_name(x):
    if x in ['HINO ZY1EWPD-XS','HINO ZY1EWRD-XS','HINO ZY1EWRN-XS']:
        msg = 'HINO ZY'
    elif x in ['HINO ZS1EPPD-XS']:
        msg = 'HINO ZS'
    else:
        msg = retext_equipment_name_t1(x)
    return msg

In [9]:
def company_to_kontrak(x):
    if x == 'CV. SENTOSA ABADI' or x == 'PT. SENTOSA ABADI MINING':
        msg = 'Mining BDM'
    elif x == 'CV. Adil Prima Perkasa':
        msg = 'Mining KBM'
    else:
        msg = 'Tidak Diketahui'
    return msg

In [10]:
# fungsi membuat rentang tanggal
def create_date_range_dataframe(start_year: int, end_year: int):
    # Tentukan tanggal awal dan akhir berdasarkan input tahun
    start_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    
    # Buat list untuk menyimpan rentang tanggal
    date_range = []

    # Hitung jumlah hari dari tanggal awal hingga tanggal akhir
    delta = end_date - start_date

    # Tambahkan setiap tanggal dalam rentang tersebut ke dalam list
    for i in range(delta.days + 1):
        date = start_date + timedelta(days=i)
        date_range.append(date)

    # Buat DataFrame dari list rentang tanggal
    date_df = pd.DataFrame({'date': date_range})

    # Ubah format tanggal menjadi 'YYYY-MM-DD'
    date_df['date'] = date_df['date'].apply(lambda x: str(x).split()[0])

    return date_df

In [11]:
from datetime import datetime, timedelta

# Tentukan tanggal awal (1 Januari 2023)
start_date = datetime(2023, 1, 1)

# Tentukan tanggal hari ini
today = datetime.today()

# Buat list untuk menyimpan rentang tanggal
date_range = []

# Hitung jumlah hari dari tanggal awal hingga hari ini
delta = today - start_date

# Tambahkan setiap tanggal dalam rentang tersebut ke dalam list
for i in range(delta.days + 1):
    date = start_date + timedelta(days=i)
    date_range.append(date)

date = pd.DataFrame({'date':date_range})
date['date'] = date['date'].apply(lambda x: str(x).split()[0])
date['month'] = pd.to_datetime(date['date']).dt.strftime('%m/%y')

_ = []
for i in date.month.unique().tolist():
    n = date[date.month==i]
    n['count_day'] = len(n)
    _.append(n)

date = pd.concat(_)

In [12]:
# Definisikan fungsi untuk menghitung jumlah nilai unik
def unique_count(series):
    return len(np.unique(series))

# **ETL : Extracting, Transforming, Loading**

## Maintenance.Request

In [13]:
path_mr = '/Users/dickyaryanto/Documents/PT Sentosa Abadi Mining/Data/maintenance request'
df2 = read_file(path_mr)

df2['Broken Date'] = df2['Broken Date'].apply(lambda x: str(x))
df2['Request Date'] = df2['Request Date'].apply(lambda x: str(x))
df2['Scheduled Date'] = df2['Scheduled Date'].apply(lambda x: str(x))
df2['Completion Date'] = df2['Completion Date'].apply(lambda x: str(x))

def transformasi_nan_format(x):
    con = str(x)
    if con == 'NaT' or con == 'nan':
        msg = np.NaN
    else:
        msg = x
    return msg

df2['Broken Date'] = df2['Broken Date'].apply(transformasi_nan_format)
df2['Request Date'] = df2['Request Date'].apply(transformasi_nan_format)
df2['Scheduled Date'] = df2['Scheduled Date'].apply(transformasi_nan_format)
df2['Completion Date'] = df2['Completion Date'].apply(transformasi_nan_format)

def completion_end_day(broken, done):
    con = str(done)
    if con == 'nan' or con == 'NaN' or con == np.NaN:
        date = str(broken).split()[0]
        date_time = date + ' 17:00:00'
        con2 = date_time.split()[0]
        if con2 == 'nan':
            date_time = np.NaN
        else:
            date_time = date_time
    else:
        date_time = done
    return date_time

df2['Completion Date'] = df2.apply(lambda x: completion_end_day(x['Broken Date'], x['Completion Date']), axis=1)
df2['Scheduled Date'] = df2.apply(lambda x: completion_end_day(x['Broken Date'], x['Completion Date']), axis=1)

df2 = df2.ffill()

path_p = '/Users/dickyaryanto/Documents/PT Sentosa Abadi Mining/Data/stock picking'

picking = read_file(path_p)

df2['Request Date'] = pd.to_datetime(df2['Request Date'])
df2 = df2[df2['Request Date'].dt.year>=2023]

# merge data df2 terkecuali berstatus Cancelles
def get_service(x):
    if x == '':
        msg = 'Service'
    else:
        msg = x
    return msg

df2['Picking Line/Status'] = df2['Picking Line/Status'].apply(get_service)
df2 = df2[df2['Picking Line/Status'].isin(['Service','Ready','Done'])]
df2 = df2.merge(picking[['Reference','Stock Moves/Product','Stock Moves/Quantity Done','Stock Moves/Unit of Measure']], left_on='Picking Line/Reference', right_on='Reference', how='left')

# refill data
df2_1 = df2[df2['Picking Line/Status']=='Service']
df2_1['Reference'] = '-'
df2_1['Stock Moves/Product'] = '-'
df2_1['Stock Moves/Quantity Done'] = 0
df2_1['Stock Moves/Unit of Measure'] = '-'

df2_2 = df2[df2['Picking Line/Status']!='Service']
df2 = pd.concat([df2_1, df2_2])

# extract maintenance.request
df = df2.copy()
df.columns = ['_'.join(i.split()).lower() for i in df.columns]
df.drop(columns='reference', inplace=True)

# transformasi tipe data
df['request_date'] = pd.to_datetime(df['request_date'])
df = df[df.request_date.dt.year >= 2023]

df.columns=['company','spk','stage','broken','request','schedule','done','oprator','category_equipment','equipment_name',
            'equipment_code','hm','km','requirement_type','type_maintenance','category_maintenance','picking_reference','picking_status','note','description','qty','uom']
df['oprator_name'] = df.oprator.apply(lambda x: str(x).split('] ')[-1])
df = df[df.spk!='D85-06']
df = df[df.spk!='PC200-12']
df['spk'] = df.spk.apply(lambda x: int(str(x).split('/')[-1]))
df.sort_values('spk', ascending=False, inplace=True)
df['spk'] = df['spk'].astype(str)

def get_description(x):
    if x == False:
        msg = 'Service Only'
    else:
        msg = x
    return msg

df['description'] = df['description'].apply(get_description)
df['uom'] = df['uom'].apply(lambda x: str(x).upper())


In [14]:
df['broken'] = pd.to_datetime(df['broken'])
df['request'] = pd.to_datetime(df['request'])
df['schedule'] = pd.to_datetime(df['schedule'])
df['done'] = pd.to_datetime(df['done'])
df['hm'] = df['hm'].astype(float)
df['km'] = df['km'].astype(float)

_ = df[df.category_equipment==False]
_['category_equipment'] = 'Tidak Diketahui'
_['equipment_name'] = 'Tidak Diketahui'
_['equipment_code'] = 'Tidak Diketahui'
__ = df[df.category_equipment!=False]
df = pd.concat([_, __]).sort_values('spk', ascending=False)

# apply function
df['equipment_name'] = df['equipment_name'].apply(retext_equipment_name)
df['category_maintenance'].replace('TRYE', 'TYRE', inplace=True)
df['category_maintenance'].replace('TRYE (Archive)', 'TYRE', inplace=True)
df['category_maintenance'].replace('TYRE-PENGGANTIAN', 'TYRE', inplace=True)
df['category_maintenance'].replace('TYRE-PERBAIKAN', 'TYRE', inplace=True)

# buat table category maintenance
tbl_cm = df[df.category_maintenance!=False][['category_maintenance','description']].drop_duplicates()

def get_correct_category_maintenance(x):
    n = tbl_cm[tbl_cm.description == x]
    if len(n) == 0:
        try:
            n = tbl_cm.copy()
            n['score'] = n.description.apply(lambda i: similairty_sentence(x, i))
            n = n.sort_values('scor', ascending=False)
            msg = n['category_maintenance'].uniqeu().tolist()[0]
        except:
            msg = x
    else:
        msg = n['category_maintenance'].unique().tolist()[0]
    return msg 

df_ = df[df.category_maintenance!=False]
df__ = df[df.category_maintenance==False]

df__['category_maintenance'] = df__.description.apply(get_correct_category_maintenance)
df = pd.concat([df_, df__])

## Product.Template

In [15]:
### extract product.template
path_pt = '/Users/dickyaryanto/Documents/PT Sentosa Abadi Mining/Data/product template'
product = read_file(path_pt)
product.columns = ['_'.join(i.lower().split()) for i in product.columns]
product['unit_of_measure'] = product['unit_of_measure'].apply(lambda x: str(x).upper())
product['name2'] = '#' + product['name'] + '#' + product['unit_of_measure']
product.drop_duplicates(subset='product', inplace=True)

# merge data product untuk dapatkan cost
df = pd.merge(df, product[['name', 'product','cost']], left_on='description', right_on='product', how='left').drop(columns='product')
df['cost'].fillna(0, inplace=True)
df['cost'] = df['cost'].astype(float)
df['qty'] = df['qty'].astype(float)

df['description'] = df['description'].apply(lambda x: str(x))
df['qty'] = df['qty'].apply(lambda x: str(x))
df['cost'] = df['cost'].apply(lambda x: str(x))

def get_decription(x):
    con = str(x)
    if con == 'nan':
        msg='Service'
    else:
        msg = x
    return msg

def get_qty(x):
    con = str(x)
    if con == 'nan':
        msg=0
    else:
        msg = x
    return msg

df['description'] = df.description.apply(get_description)
df['qty'] = df['qty'].apply(get_qty)
df['cost'] = df['cost'].apply(get_qty)
df = df.rename(columns={'name':'description_2'})
df['description_2'] = df['description_2'].apply(lambda x: str(x))
df['description_2'] = df.description_2.apply(get_description)
df['description_2'] = df['description_2'].replace('nan', 'Service')
df['description'] = df['description'].replace('nan', 'Service')
df['uom'] = df['uom'].replace('NAN', '-')
df['cost'] = df['cost'].astype(float)
df['qty'] = df['qty'].astype(float)

# conditional
a = df[df.equipment_name=='HONGYAN KINKAN430']
b = df[df.equipment_name!='HONGYAN KINKAN430']
a.requirement_type.replace('External', 'Internal', inplace=True)
df = pd.concat([a, b])

## Standar Pengukuran

In [16]:
path_standar_ukur = '/Users/dickyaryanto/Documents/PT Sentosa Abadi Mining/Data/standar ukur/'

tbl_jarak = read_file(path_standar_ukur, 'tbl jarak move type')
tbl_kapasitas = read_file(path_standar_ukur, 'tbl kapasitas dt')
tbl_waktu = read_file(path_standar_ukur, 'tbl waktu move type')
tbl_periodical = read_file(path_standar_ukur, 'tbl periodical unit')

## Production.Timesheet

In [17]:
### extract production.timesheet
path_ts = '/Users/dickyaryanto/Documents/PT Sentosa Abadi Mining/Data/production timesheet'
df2 = read_file(path_ts)

df2.columns = ['_'.join(i.split()).lower() for i in df2.columns]
df2 = df2[df2.date.dt.year >= 2023]

# df2 = df2[df2['date']!=pd.to_datetime(str(pd.to_datetime(datetime.now())).split()[0])]
df2['hm_realtime_start'] = df2['hm_realtime_start'].apply(lambda x: str(x).split('.')[0])
df2['hm_realtime_end'] = df2['hm_realtime_end'].apply(lambda x: str(x).split('.')[0])

def get_true_time(date, time):
    time = str(time)
    if len(time) == 2:
        time = str(time) + ':00:00'
    else:
        time = '0' + str(time) + ':00:00'
        
    date = str(date).split()[0]
    msg = date + ' ' + time

    try:
        a = pd.to_datetime(msg)
    except:
        msg = 'error'
    
    return msg

df2['date_engine_start'] = df2.apply(lambda x: get_true_time(x['date'], x['hm_realtime_start']), axis=1)
df2['date_engine_end'] = df2.apply(lambda x: get_true_time(x['date'], x['hm_realtime_end']), axis=1)

# split data
df2_err = df2[(df2['date_engine_start']=='error') & (df2['date_engine_end']=='error')]
df2 = df2[(df2['date_engine_start']!='error') & (df2['date_engine_end']!='error')]

df2['date_engine_start'] = pd.to_datetime(df2['date_engine_start'])
df2['date_engine_end'] = pd.to_datetime(df2['date_engine_end'])

# cleaning condition memungkinkan tidak akan digunakan bila data bersih
df2_err['actual_engine'] = df2_err['hm_realtime_end'].astype(float) - df2_err['hm_realtime_start'].astype(float)
def retext_time(x):
    time = str(x)
    if len(time) == 2:
        time = str(time) + ':00:00'
    else:
        time = '0' + str(time) + ':00:00'
    return time

def get_time_from_err(x):
    start = 7
    end = start + x

    start_time = retext_time(str(start))
    end_time = retext_time(str(end).split('.')[0])

    msg = {'start':start_time, 'end':end_time}
    return msg

df2_err['hm_realtime_end'] = df2_err['actual_engine'].apply(lambda x: get_time_from_err(x)['end'])
df2_err['hm_realtime_start'] = df2_err['actual_engine'].apply(lambda x: get_time_from_err(x)['start'])

df2_err['date_engine_start'] = df2_err['date'].apply(lambda x: str(x).split()[0]) + ' ' + df2_err['hm_realtime_start']
df2_err['date_engine_end'] = df2_err['date'].apply(lambda x: str(x).split()[0]) + ' ' + df2_err['hm_realtime_end']

df2_err['date_engine_start'] = pd.to_datetime(df2_err['date_engine_start'])

_ = []
for i in df2_err.index:
    n = df2_err[df2_err.index==i]
    try:
        n['date_engine_end'] = pd.to_datetime(n['date_engine_end'])
    except:
        n['date_engine_end'] = 'error'
    _.append(n)

df2_err = pd.concat(_)

In [18]:
# transform data
error_actual_date = df2_err[df2_err.date_engine_end=='error']
error_actual_date = error_actual_date.merge(df[['equipment_name','equipment_code']].drop_duplicates(), left_on='equipment', right_on='equipment_code', how='left').drop(columns='equipment_code')
error_actual_date = error_actual_date.merge(tbl_kapasitas[['equipment_name','rerata_muatan']], on='equipment_name', how='left')
error_actual_date['pred_retase'] = np.round((error_actual_date['component_line/net_weight/voll']/1000) / error_actual_date.rerata_muatan)
error_actual_date = error_actual_date.merge(tbl_waktu, on='move_type', how='left')
error_actual_date['actual_engine'] = error_actual_date.pred_retase * error_actual_date.waktu
error_actual_date = error_actual_date.drop(columns=['pred_retase','equipment_name','rerata_muatan','waktu'])

# concat data
df2_err = df2_err[df2_err.date_engine_end!='error']
df2 = pd.concat([df2, df2_err])

df2['date_engine_end'] = pd.to_datetime(df2['date_engine_end'])
df2['date_engine_start'] = pd.to_datetime(df2['date_engine_start'])
df2['actual_engine'] = df2.date_engine_end - df2.date_engine_start
df2['actual_engine'] = df2['actual_engine'].apply(timedelta_to_hour)

df2 = pd.concat([df2, error_actual_date])
# df2 = df2[df2.state!='Draft']

In [19]:
df2['component_line/out_weight'].fillna(0, inplace=True)
df2['component_line/net_weight/voll'].fillna(0, inplace=True)
df2['production'].replace('ROAD MAINTENANCE', 'Road Maintenance', inplace=True)

## Maintenance.Equipment

In [20]:
path_eq = '/Users/dickyaryanto/Documents/PT Sentosa Abadi Mining/Data/maintenance equipment'
tbl_equipment2 = read_file(path_eq)
tbl_equipment2['equipment_code'] = tbl_equipment2.Name.apply(lambda x: str(x) + '/') + tbl_equipment2['Serial Number'].apply(lambda x: str(x))
tbl_equipment2.columns = ['_'.join(i.lower().split()) for i in tbl_equipment2.columns.tolist()]

# dapatkan tabel equipment 
tbl_eq = tbl_equipment2[['company','equipment_category','equipment_model','name']]
tbl_eq['equipment_model'] = tbl_eq['equipment_model'].apply(retext_equipment_name)
tbl_eq.columns = ['company','category_equipment','equipment_name','name']
tbl_eq['equipment_name'] = tbl_eq.equipment_name.apply(retext_equipment_name)

## Purchase.Order

### 1. PO Node3

In [21]:
path_po = '/Users/dickyaryanto/Documents/PT Sentosa Abadi Mining/Data/purchase order'
po = read_file(path_po)

po.columns = ['_'.join(i.lower().split()) for i in po.columns]
po['total'] = po['order_lines/quantity'] * po['order_lines/unit_price']
po = po.ffill()
po.drop(columns=['source_document','purchase_requests','order_lines/description'], inplace=True)
po = po.rename(columns={
    'order_reference':'no_po',
    'purchase_request':'no_pr',
    'order_lines/unit_price':'price',
    'order_lines/quantity':'qty',
    'order_lines/unit_of_measure':'uom',
    'order_lines/disc.%':'discount',
    'order_lines/product':'description',
    'receipt_date':'date'
})

po['discount'] = po.discount / 100
po['total'] = po.total - (po.total * po.discount)
po['total'] = po.total + (po.total * 0.11)
po['no_spk'] = po['no_spk'].apply(lambda x: str(str(x).split('/')[-1]))
po = po[['date','no_pr','no_po','no_spk','description','qty','uom','price','discount','total']]
po['sumber'] = 'PO SA'

### 2. PO Selain Dari Node3

In [22]:
### extract data PO diluar Node3 ambil dari Google Drive 
url = "https://docs.google.com/spreadsheets/d/1Ay0TNz8yPhdNzr4KhGmNy1QENGJS6B-Z/export?format.xlsx"

try:
    os.remove('po_sam.xlsx')
except:
    None
    
output_filename = "po_sam.xlsx"

# get the data from spreadsheet
response = requests.get(url)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)

# transformasi nama columns
po_sam = pd.read_excel('po_sam.xlsx', sheet_name='Sheet1')
po_sam.columns = ['_'.join(i.lower().split()) for i in po_sam.columns]
po_sam['spk'].fillna('kosong', inplace=True)
po_sam = po_sam[po_sam.spk!='kosong']
po_sam = po_sam[['date','spk','wo','customer','id','unnamed:_10','unnamed:_11','unnamed:_12','unnamed:_13','unnamed:_14','remark']]
po_sam.columns = ['date','spk_sam','spk_sa','customer','name','description','qty','discount','price','total','stage']

def get_same_text(text1):
    n = tbl_equipment2.copy()
    n['similairity'] = n.name.apply(lambda x: similairty_sentence(text1, x))
    n.sort_values('similairity', ascending=False, inplace=True)
    n = n.head(1)

    score = n['similairity'].unique().tolist()[0]
    if score >= 0.9:
        try:
            cat_eq = n['equipment_category'].unique().tolist()[0]
            eq_model = n['equipment_model'].unique().tolist()[0]
            name = n['name'].unique().tolist()[0]
            id = n['serial_number'].unique().tolist()[0]
            msg = {'cat_eq':[cat_eq],'eq_model':[eq_model],'name':[name],'id':[id]}
        except:
            msg = {'cat_eq':[np.NaN],'eq_model':[np.NaN],'name':[text1],'id':[np.NaN]}
    else:
        msg = {'cat_eq':[np.NaN],'eq_model':[np.NaN],'name':[text1],'id':[np.NaN]}
        
    return msg

def get_same_text_category_maintenance(text1):
    n = df.copy()
    n = n[['category_maintenance','description']]
    n.drop_duplicates(inplace=True)
    n['similairity'] = n.description.apply(lambda x: similairty_sentence(text1, x))
    n.sort_values('similairity', ascending=False, inplace=True)
    n = n.head(1)

    score = n['similairity'].unique().tolist()[0]
    if score >= 0.8:
        try:
            msg = n['category_maintenance'].unique().tolist()[0]
        except:
            msg = np.NaN
    else:
        msg = np.NaN
        
    return msg

po_sam.description.fillna('HMSI', inplace=True)
po_sam.discount.fillna(0, inplace=True)
po_sam['no_pr'] = '-'
po_sam['uom'] = '-'
po_sam['no_po'] = '-'
po_sam = po_sam[['date','no_pr','spk_sam','spk_sa','description','qty','uom','price','discount','total']].rename(columns={'spk_sa':'no_spk','spk_sam':'no_po'})
po_sam['no_spk'] = po_sam.no_spk.apply(lambda x: ''.join(str(x).split('.')))
po_sam['sumber'] = 'PO SAM'

In [23]:
po = pd.concat([po, po_sam])

## Produksi Actual Site

### 1. Absen Driver

In [24]:
url = "https://docs.google.com/spreadsheets/d/1CDBo-9hz0-Vc7e-hgjG1qdIZ07-PEdLlMatzssJhTJU/export?format.xlsx"
try:
    os.remove('absen_driver.xlsx')
except:
    None
    
output_filename = 'absen_driver.xlsx'

# get the data from spreadsheet
response = requests.get(url)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)

# read data absen driver dari tbl produksi actual site
absen_driver = pd.read_excel('absen_driver.xlsx', sheet_name='Absen Driver')
absen_driver = absen_driver[1:]
absen_driver.columns = ['_'.join(i.lower().split()) for i in absen_driver.columns]
absen_driver = absen_driver[['nama_driver','id_unit','project','date','jam_keluar','jam_masuk','total_jam_kerja','retase','keterangan']]
absen_driver.columns = ['driver','name','project/name','date','jam_keluar','jam_masuk','total_jam_kerja','retase','keterangan']

absen_driver['driver'].fillna('Tidak Diketahui', inplace=True)
absen_driver['name'] = absen_driver.name.apply(lambda x: 'DT-' + x[-3:])
absen_driver['date'] = absen_driver['date'].apply(lambda x: str(x).split()[0])
absen_driver['total_jam_kerja'].fillna('-', inplace=True)

### 2. Grade

In [25]:
url = "https://docs.google.com/spreadsheets/d/12_BTs8ftDwcp0DCva7wTZzLuzaE9nkKrgJ_onoqMbDM/export?format.xlsx"

try:
    os.remove('grade_report.xlsx')
except:
    None
    
output_filename = 'grade_report.xlsx'

# get the data from spreadsheet
response = requests.get(url)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)

In [26]:
# read data grade
df_grade = pd.read_excel('grade_report.xlsx', sheet_name='Laporan Grade')
df_grade = df_grade[1:]
df_grade = df_grade[['Date','AP6','AP7','AP8','AP9','AP10','AP11','KBM C','KMB D','KBM E','TUJUAN PABRIK',
          'TONASE','% GRADE (LAB. SM)','Unnamed: 14','Unnamed: 15','% GRADE (LAB. Feeding)','Unnamed: 17','Unnamed: 18']]
df_grade.columns = ['date','ap6','ap7','ap8','ap9','ap10','ap11','kmb_c','kmb_d','kmb_e','pabrik','produksi','sm_ni','sm_fe','sm_si','f_ni','f_fe','f_si']

# cleaning data
df_grade['date'] = df_grade['date'].apply(lambda x: str(x).split()[0])
df_grade['pabrik'].fillna('-', inplace=True)
df_grade.fillna(0, inplace=True)
df_grade = df_grade[df_grade.pabrik!='-']

df_grade['sm_ni'] = df_grade['sm_ni'].apply(lambda x: '.'.join(str(x).split(',')))
df_grade['sm_fe'] = df_grade['sm_fe'].apply(lambda x: '.'.join(str(x).split(',')))
df_grade['sm_si'] = df_grade['sm_si'].apply(lambda x: '.'.join(str(x).split(',')))

df_grade['sm_ni'] = df_grade['sm_ni'].apply(lambda x: float(str(x).split('%')[0]))
df_grade['sm_fe'] = df_grade['sm_fe'].apply(lambda x: float(str(x).split('%')[0]))
df_grade['sm_si'] = df_grade['sm_si'].apply(lambda x: float(str(x).split('%')[0]))
df_grade['produksi'] = df_grade['produksi'].apply(lambda x: float(''.join(str(x).split(','))))

grade = df_grade[['date','ap6','ap7','ap8','ap9','ap10','ap11','kmb_c','kmb_d','kmb_e','pabrik','produksi','sm_ni','sm_fe','sm_si']]
df_grade.to_excel('./Report/grade control.xlsx', index=False)

### 3. Accdient Rate

In [27]:
url = "https://docs.google.com/spreadsheets/d/19y9xZ3L3AMYGDZOVG_6Q5hNX7HFQSW__tf6YXX_Ojgk/export?format.xlsx"
try:
    os.remove('accident_rate.xlsx')
except:
    None
    
output_filename = 'accident_rate.xlsx'

# get the data from spreadsheet
response = requests.get(url)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)

# read data absen driver dari tbl produksi actual site
accident = pd.read_excel('accident_rate.xlsx')
accident = accident[1:]
accident.columns = ['timestamps','id','date','no_ba','kat_kecelakaan','tempat',
                    'name','departemen','employee','umur','estimate','keterangan','id_user','user']
accident = accident.drop(columns=['id_user','user'])

### 4. Fuel & Fuel Cost

In [28]:
# buat kolom bulan di dataframe date
date['month'] = pd.to_datetime(date.date).dt.strftime('%m/%y')

# dapatkan nilai fuel
url_konsumsi = "https://docs.google.com/spreadsheets/d/1Q9nZYOoRZZL_PB6wUPDyBtzuWJv2qZXpbXYlt7wrGho/export?format.xlsx"

try:
    os.remove('fuel_comsumption.xlsx')
except:
    None
    
output_filename = 'fuel_comsumption.xlsx'

# get the data from spreadsheet
response = requests.get(url_konsumsi)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)

# read data
fuel = pd.read_excel(output_filename)
fuel = fuel[fuel.index>0].reset_index(drop=True).drop(columns='CODE USER')
fuel.columns = ['_'.join(i.lower().split()) for i in fuel.columns]
fuel = fuel.rename(columns={'id_unit':'name'})
fuel = fuel.merge(tbl_eq[['category_equipment','equipment_name','name']].drop_duplicates(), on='name', how='left')
fuel['timestamps'].fillna('kosong', inplace=True)
fuel = fuel[fuel.timestamps != 'kosong']
fuel['move_type'].fillna('Tidak Diketahui', inplace=True)
fuel['hm_saat_isi_fuel'].fillna(0, inplace=True)
fuel['km_saat_isi_fuel'].fillna(0, inplace=True)

fuel['id'] = fuel['id'].astype(int)
id_bulanan = list(np.arange(589, 2554))

# buat kondisi laporan karena laporan pemakaian fuel sebelumnya dalam bulanan 
def get_id_condtional(x):
    if x in id_bulanan: 
        msg = 'Konsumsi Bulanan'
    else:
        msg = 'Konsumsi Harian'
    return msg
    
fuel['report_type'] = fuel['id'].apply(get_id_condtional)
fuel = fuel[['timestamps','id','report_type','date','project','production','move_type','category_equipment','equipment_name',
             'name','hm_saat_isi_fuel','km_saat_isi_fuel','fuel_consumption','user']]

fuel['date'] = pd.to_datetime(fuel['date']).dt.date
fuel = fuel.groupby(['date','report_type','category_equipment','equipment_name','name'])[['fuel_consumption']].sum().reset_index().rename(columns={'fuel_consumption':'fuel'})
fuel_m = fuel[fuel.report_type=='Konsumsi Bulanan']
fuel_m['month'] = pd.to_datetime(fuel_m.date).dt.strftime('%m/%y')

_ = []
for name in tqdm(fuel_m.name.unique().tolist()):
    n = fuel_m[fuel_m.name == name]
    for month in n.month.unique().tolist():
        m = n[n.month==month]

        cat_eq = m.category_equipment.unique().tolist()[0]
        eq_name = m.equipment_name.unique().tolist()[0]
        name = name
        count_day = len(date[date.month==month])
        fuel_per_day = m.fuel.sum()/count_day
        
        msg = date[date.month==month][['date']].merge(m, on='date', how='left').drop(columns=['report_type','month'])
        msg['category_equipment'] = cat_eq
        msg['equipment_name'] = eq_name
        msg['name'] = name
        msg['fuel'] = fuel_per_day
        _.append(msg)
fuel_m = pd.concat(_)
fuel_h = fuel[fuel.report_type!='Konsumsi Bulanan'].drop(columns=['report_type'])
fuel = pd.concat([fuel_m, fuel_h])
fuel['date'] = pd.to_datetime(fuel['date']).dt.date

# dapatkan data harga fuel
url_price_fuel = "https://docs.google.com/spreadsheets/d/1BJC_Wc58T9LxnW7ZLimpNIA_1ZRijaaDF2DiJe6jlPs/export?format.xlsx"
try:
    os.remove('fuel_price.xlsx')
except:
    None
    
output_filename = 'fuel_price.xlsx'
# get the data from spreadsheet
response = requests.get(url_price_fuel)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)

# read data
fuel_price = pd.read_excel(output_filename)
fuel_price.columns = ['date','fuel_type','price','pbb_kb']
fuel_price['price2'] = fuel_price.price + fuel_price.pbb_kb
fuel_price.sort_values('date', ascending=True, inplace=True)
fuel_price['date'] = fuel_price['date'].dt.date
left = date.drop(columns='month')
left['date'] = pd.to_datetime(left['date']).dt.date
fuel_price = left.merge(fuel_price, on='date', how='outer')
fuel_price = fuel_price.bfill().ffill()

# gabungkan data harga fuel 
fuel = fuel.merge(fuel_price[['date','price2']], on='date', how='left')
fuel['fuel_cost'] = fuel.price2 * fuel.fuel

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 230/230 [00:01<00:00, 126.22it/s]


# **EDA : Exploratory Data Analyst - 1**

## Task 1.0 : DASHBOARD

Berapa waktu rata-rata yang dibutuhkan untuk team Workshop memperbaiki setiap request dari setiap crew produksi/transportasi?

In [29]:
t1 = df.copy()
t1['t_broken_done'] = t1.done - t1.broken
t1['t_schedule_done'] = t1.done - t1.schedule
t1['requirement_type'].replace('', 'Internal', inplace=True)

t1_in = t1[t1.requirement_type=='Internal']
t1_in['no_po'] = '-'

t1_ex = t1[t1.requirement_type=='External']
t1_ex['spk'] = t1_ex['spk'].apply(lambda x: str(x))
t1_ex.drop(columns=['description','qty','uom','description_2','cost'], inplace=True)

# pertemukan semua yang external kepada po sam
po_spk = po[po.no_spk!='False']
t1_ex = t1_ex.drop_duplicates(subset='spk').merge(po_spk[['no_spk','no_po','description','qty','uom','total']], how='left', left_on='spk', right_on='no_spk')
t1_ex = t1_ex.rename(columns={'total':'cost'})
t1_ex['uom'] = t1_ex['uom'].apply(lambda x: str(x).upper())
t1_ex['no_po'].fillna('-', inplace=True)
t1_ex['description_2'] = t1_ex.description.apply(get_description)
t1_ex['description_2'].fillna('Service', inplace=True)
t1_ex = t1_ex[t1_in.columns]
t1_ex['cost'].fillna(0, inplace=True)
t1_ex['cost'] = t1_ex.cost / t1_ex.qty

t1 = pd.concat([t1_in, t1_ex])
t1 = t1.sort_values('spk', ascending=False).reset_index(drop=True)

# create function to transform timedelat to minute
def timedelta_to_minute(timedelta_str):
    # Konversi timedelta string ke timedelta object
    timedelta_obj = pd.to_timedelta(timedelta_str)
    
    # Hitung total detik dalam timedelta
    total_seconds = timedelta_obj.total_seconds()
    
    # Konversi total detik ke jam
    total_minute = total_seconds / 60
    
    return total_minute

t1['t_broken_done'] = t1['t_broken_done'].apply(timedelta_to_minute)
t1['t_schedule_done'] = t1['t_schedule_done'].apply(timedelta_to_minute)
t1['t_broken_done'].fillna(0, inplace=True)
t1['t_schedule_done'].fillna(0, inplace=True)
t1 = t1[t1.stage != 'Cancel']
t1['cost'] = t1.cost * t1.qty

def get_time(x):
    hours = int(x // 60)
    remaining_minutes = int(x % 60)
    formated_time = f"{hours} Hour {remaining_minutes} Minute"
    formated_time = pd.to_timedelta(formatted_time)
    return formated_time

t1['number'] = 1
t1['category_maintenance'].replace('TRYE', 'TYRE', inplace=True)
t1['name'] = t1.equipment_code.apply(lambda x: str(x).split('/')[0])
t1['equipment_name'] = t1['equipment_name'].apply(lambda x: ' '.join(str(x).split()))
t1['equipment_name'] = t1['equipment_name'].apply(lambda x: retext_equipment_name(x))
t1 = t1[t1.equipment_name!='NISSAN CWA']

# buat fungsi untuk klasfikasi jenis kerusakan secara lebih meluas 
def retext_category_maintenance(x):
    try:
        msg = ' '.join(x.split('] ')[-1].split()[:1]).upper()
    except:
        msg = x
    return msg
    
t1['category_maintenance'] = t1['category_maintenance'].apply(retext_category_maintenance)

# buat fungsi untuk klasfikasi jenis kerusakan secara lebih meluas 
def retext_description_maintenance(x):
    try:
        msg = ' '.join(x.split('] ')[-1].split()[:2]).upper()
    except:
        msg = x
    return msg
t1['description_3'] = t1.description_2.apply(retext_description_maintenance)

In [30]:
check = t1.copy()
check['date'] = pd.to_datetime(check.broken).dt.date
check['date'] = check['date'].apply(lambda x: str(x).split()[0])
check = check[(check.category_maintenance=='TYRE') & (check.description_3.isin(['SERVICE','BAN LUAR']))]

In [31]:
## TEMUKAN KETERANGAN WORKSHOP UNTUK DT TERDAFTAR BERIKUT

# list_dt = ['DT-316','DT-391','DT-398','DT-389','DT-318','DT-369']
# check = t1.copy()
# check['date'] = check.broken.apply(lambda x: str(x).split()[0])
# check = check[check.date.isin(['2024-07-09'])]
# check = check[check.name.isin(list_dt)]
# check = check.drop_duplicates(subset='spk')[['spk','stage','oprator','broken','done','equipment_name','name','t_broken_done']].rename(columns={'t_broken_done':'waktu maintenance'})
# check['waktu maintenance'] = check['waktu maintenance']/60

In [32]:
t1.to_excel('./Report/task1.xlsx', index=False)

## Task 1.1 : DASHBOARD

In [33]:
t1_1 = t1.copy()

t1_1['date_broken'] = pd.to_datetime(t1_1.broken).dt.date
t1_1['category_maintenance'].fillna('Tidak Diketahui', inplace=True)

# retext category maintenance lebih umum
t1_1['category_maintenance'] = t1_1['category_maintenance'].apply(retext_category_maintenance)

broken = t1_1[['stage','broken','requirement_type','type_maintenance','category_maintenance','description_3','category_equipment','equipment_name','spk']]
broken['date'] = broken.broken.dt.date
broken = broken.drop_duplicates(subset='spk')
broken = broken.groupby(['date','requirement_type','type_maintenance','category_maintenance','description_3','category_equipment','equipment_name'])[['stage']].count().reset_index().rename(columns={'stage':'values'})
broken['status'] = 'Broken'

done = t1_1[['stage','done','requirement_type','type_maintenance','category_maintenance','description_3','category_equipment','equipment_name','spk']]
done['date'] = done.done.dt.date
done = done.drop_duplicates(subset='spk')
done = done[done.stage=='DONE']
done = done.groupby(['date','requirement_type','type_maintenance','category_maintenance','description_3','category_equipment','equipment_name'])[['stage']].count().reset_index().rename(columns={'stage':'values'})
done['status'] = 'Done'

t1_1 = pd.concat([broken, done])
t1_1 = t1_1[pd.to_datetime(t1_1.date).dt.year >= 2023]
t1_1 = t1_1.rename(columns={'date':'request'})
t1_1.to_excel('./Report/task1_1.xlsx', index=False)

## Task 2.1 : DASHBOARD

KPI Detail

In [34]:
# jam_operasional = df2.groupby(['date','name','company','project','production','move_type','equipment'])[['hm_engine_start','hm_engine_end','component_line/out_weight','component_line/in_weight']].sum().reset_index()
# jam_operasional['produksi'] = (jam_operasional['component_line/in_weight'] - jam_operasional['component_line/out_weight'])/1000
# jam_operasional.drop(columns=['component_line/out_weight','component_line/in_weight'], inplace=True)
# jam_operasional['equipment'] = jam_operasional.equipment.apply(lambda x: str(x).split('/')[0])
# jam_operasional = jam_operasional.rename(columns={'name':'ts','equipment':'name'})

# _ = df[['category_equipment','equipment_name','equipment_code']].drop_duplicates()
# _['equipment_code'] = _['equipment_code'].apply(lambda x: str(x).split('/')[0])
# _ = _.rename(columns={'equipment_code':'name'})

# jam_operasional = jam_operasional.merge(_[['category_equipment','equipment_name','name']].drop_duplicates(), on='name')
# jam_operasional = jam_operasional[['date','ts','company','project','production','move_type','category_equipment',
#                                    'equipment_name','name','hm_engine_start','hm_engine_end','produksi']]
# jam_operasional['date'] = pd.to_datetime(jam_operasional.date).dt.date

In [35]:
jam_operasional = df2.copy()
jam_operasional['produksi'] = (jam_operasional['component_line/in_weight'] - jam_operasional['component_line/out_weight'])/1000
jam_operasional['equipment'] = jam_operasional.equipment.apply(lambda x: str(x).split('/')[0])
jam_operasional = jam_operasional.rename(columns={'name':'ts','equipment':'name'})
jam_operasional['date_engine_start'] = jam_operasional['date_engine_start'].apply(lambda x: str(x))
jam_operasional['date_engine_end'] = jam_operasional['date_engine_end'].apply(lambda x: str(x))

# buat fungsi untuk join 2 nilai data
def join_text(x):
    msg = ';'.join(x)
    return msg

# buat fungsi untuk menghitung retase dari jumlah timesheet
def count_retase(x):
    msg = len(x.split(';'))
    return msg

# re-text hm jika > 10
def in_scale(x):
    if x > 10:
        msg = 10
    else:
        msg = x
    return msg

# olah data gabungkan untuk ts berbeda namun masih dalam 1 DT di hari yang sama
jam_operasional = jam_operasional.groupby(['date','company','project','production','move_type','name']).agg({
    'date_engine_start':join_text,
    'date_engine_end':join_text,
    'ts':join_text,
    'hm_engine_start':np.sum,
    'hm_engine_end':np.sum,
    'produksi':np.sum
}).reset_index()

# count jml retase dari jml ts 
jam_operasional['retase'] = jam_operasional.ts.apply(count_retase)

In [36]:
def get_start_engine(x):
    x = min(x.split(';'))

    con = int(x.split()[-1].split(':')[0])
    if con == 0:
        msg = x.split()[0] + ' 07:00:00'
    else:
        msg = x
    return msg

jam_operasional['date_engine_start'] = jam_operasional['date_engine_start'].apply(get_start_engine)

In [37]:
def get_end_engine(x):
    try:
        x = x.split(';')
        try:
            x.remove('error')
        except:
            x = x

        x = min(x)
        con = int(x.split()[-1].split(':')[0])
        if con == 0:
            msg = x.split()[0] + ' 17:00:00'
        else:
            msg = x
    except:
        msg = 'error'
    return msg

jam_operasional['date_engine_end'] = jam_operasional.date_engine_end.apply(get_end_engine)

In [38]:
# dapatkan hm dengan mengurangi hm_engine_end - hm_engine_start
jam_operasional['hm'] = np.round(jam_operasional.hm_engine_end - jam_operasional.hm_engine_start)
jam_operasional['hm'] = jam_operasional['hm'].apply(in_scale)

# rename columns
jam_operasional = jam_operasional.rename(columns={'ts':'spk_project','project':'project/name','hm':'hm_engine_actual'})

# re-columns dataframe
jam_operasional = jam_operasional[['company','spk_project','project/name','production','move_type','date','name','date_engine_start','date_engine_end','hm_engine_actual','produksi','retase']]

jam_operasional['date'] = jam_operasional['date'].apply(lambda x: str(x).split()[0])

In [39]:
jam_breakdown = df[['stage','spk','requirement_type','category_equipment','equipment_name','equipment_code','broken','done']]
jam_breakdown['name'] = jam_breakdown.equipment_code.apply(lambda x: str(x).split('/')[0])
jam_breakdown = jam_breakdown[['stage', 'spk','requirement_type', 'category_equipment', 'equipment_name','equipment_code', 'name', 'broken', 'done']]

In [40]:
jam_breakdown = jam_breakdown.rename(columns={'spk':'spk_workshop'})
jam_breakdown = jam_breakdown[jam_breakdown.stage!='Cancel']
jam_breakdown = jam_breakdown[jam_breakdown.equipment_code != False]

# fill jam done dengan waktu update apabila belum selesai diperbaiki
from datetime import datetime
def get_done_time(x, breakdown):
    con = str(x)
    if con == None or con == np.NaN or con == 'NaT':
        msg = get_work_date(breakdown)['end']
    else:
        msg = x
    return msg

In [41]:
jam_breakdown['done'] = jam_breakdown.apply(lambda x: get_done_time(x.done, x.broken), axis=1)
jam_breakdown.dropna(axis=0, inplace=True)

In [42]:
jam_breakdown['date'] = jam_breakdown.broken.apply(lambda x: str(x).split()[0])

In [43]:
# sample = {'stage': {289906: 'DONE', 289907: 'DONE'},
#  'spk_workshop': {289906: '33842', 289907: '33843'},
#  'requirement_type': {289906: 'Internal', 289907: 'Internal'},
#  'category_equipment': {289906: 'DUMP TRUCK', 289907: 'DUMP TRUCK'},
#  'equipment_name': {289906: 'HINO FM 260 Ti', 289907: 'HINO FM 260 Ti'},
#  'equipment_code': {289906: 'DT-86/JO8EUFJ67769',
#   289907: 'DT-86/JO8EUFJ67769'},
#  'name': {289906: 'DT-86', 289907: 'DT-86'},
#  'date': {289906: '2023-01-09', 289907: '2023-01-09'},
#  'broken': {289906: pd.to_datetime('2023-01-09 07:00:00'), 289907: pd.to_datetime('2023-01-09 07:00:00')},
#  'done': {289906: '2023-01-11 15:00:00', 289907: '2023-01-09 11:30:00'},
#  'actual_start_date': {289906: pd.to_datetime('2023-01-09 07:00:00'),
#   289907: pd.to_datetime('2023-01-09 07:00:00')},
#  'actual_end_date': {289906: pd.to_datetime('2023-01-09 17:00:00'),
#   289907: pd.to_datetime('2023-01-09 17:00:00')}}

# sample = pd.DataFrame(sample)

# # try

# jam_breakdown = sample.copy()

In [44]:
_ = []
for i in jam_breakdown.name.unique():
    n = jam_breakdown[jam_breakdown.name==i]
    n = n.merge(date[['date']], on='date', how='outer')

    cat_eq = n.category_equipment.unique().tolist()[0]
    eq_name = n.equipment_name.unique().tolist()[0]
    eq_code = n.equipment_code.unique().tolist()[0]

    n['category_equipment'].fillna(cat_eq, inplace=True)
    n['equipment_name'].fillna(eq_name, inplace=True)
    n['equipment_code'].fillna(eq_code, inplace=True)
    n['name'].fillna(i, inplace=True)
    n['stage'].fillna('-', inplace=True)
    n['spk_workshop'].fillna('-', inplace=True)
    n['requirement_type'].fillna('-', inplace=True)
    
    n = n.sort_values('date', ascending=True).reset_index(drop=True)
    _.append(n)

jam_breakdown = pd.concat(_)

In [45]:
jam_breakdown['actual_start_date'] = jam_breakdown['date'].apply(lambda x: get_work_date(x)['start'])
jam_breakdown['actual_end_date'] = jam_breakdown['date'].apply(lambda x: get_work_date(x)['end'])
jam_breakdown['actual_start_date'] = pd.to_datetime(jam_breakdown['actual_start_date'])
jam_breakdown['actual_end_date'] = pd.to_datetime(jam_breakdown['actual_end_date'])
jam_breakdown = jam_breakdown.drop_duplicates()

In [46]:
from datetime import datetime, timedelta

def calculate_total_breakdown_without_double(list_broken, list_done):
    # Convert string to datetime objects
    intervals = [(datetime.strptime(broken, '%Y-%m-%d %H:%M:%S'), 
                  datetime.strptime(done, '%Y-%m-%d %H:%M:%S')) 
                 for broken, done in zip(list_broken, list_done)]
    
    # Sort intervals by start time
    intervals.sort(key=lambda x: x[0])
    
    total_breakdown_time = timedelta(0)
    current_start, current_end = intervals[0]
    
    for start, end in intervals[1:]:
        if start <= current_end:
            current_end = max(current_end, end)
        else:
            total_breakdown_time += (current_end - current_start)
            current_start, current_end = start, end
    
    total_breakdown_time += (current_end - current_start)
    
    return total_breakdown_time.total_seconds() / 60

In [47]:
jam_breakdown = jam_breakdown[['stage','spk_workshop','requirement_type','category_equipment','equipment_name',
                               'equipment_code','name','date','broken','done','actual_start_date','actual_end_date']]

In [48]:
done_time_db = jam_breakdown[['name','done']].drop_duplicates()

done_time_db['done'] = done_time_db['done'].apply(lambda x: str(x))
done_time_db = done_time_db[done_time_db.done!='NaT']

done_time_db['date'] = done_time_db['done'].apply(lambda x: str(x).split()[0])

jam_breakdown['id'] = jam_breakdown.date + '#' + jam_breakdown.name
done_time_db['id'] = done_time_db.date + '#' + done_time_db.name

jam_breakdown = jam_breakdown.merge(done_time_db[['id','done']], on='id', how='left')
jam_breakdown['broken'] = jam_breakdown['broken'].apply(lambda x: str(x))
jam_breakdown['done_x'] = jam_breakdown['done_x'].apply(lambda x: str(x))
jam_breakdown['done_y'] = jam_breakdown['done_y'].apply(lambda x: str(x))

In [49]:
def fix_done_time(a, b):
    a = str(a)
    b = str(b)

    if a == 'NaT' or a == 'nan' or a == np.NaN:
        msg = b
    else:
        msg = a
    return msg

jam_breakdown['done_x'] = jam_breakdown.apply(lambda x: fix_done_time(x.done_x, x.done_y), axis=1)
jam_breakdown = jam_breakdown.drop(columns=['id','done_y']).rename(columns={'done_x':'done'})

def fix_broken_time(a, b):
    b = str(b)
    if b == 'NaT' or b == 'nan' or b == np.NaN:
        msg = 'NaT'
    else:
        msg = a
    return msg

jam_breakdown['broken'] = jam_breakdown.apply(lambda x: fix_broken_time(x['broken'], x['done']), axis=1)

In [50]:
def replace_nat(x):
    x = str(x)
    if x == 'NaT':
        msg = np.NaN
    else:
        msg = x
    return msg

def corrective_date(done, date):
    done_date = pd.to_datetime(str(done).split()[0])
    date = pd.to_datetime(date)

    if done_date < date:
        msg = np.NaN
    else:
        msg = done
    return msg

_ = []
for i in tqdm(jam_breakdown.name.unique().tolist()):
    n = jam_breakdown[jam_breakdown.name==i]
    n = n.sort_values(['date','broken','done'], ascending=True).reset_index(drop=True)
    n['broken'] = n['broken'].apply(replace_nat)
    n['done'].replace('nan', np.NaN, inplace=True)
    
    # n = n[n.index>7].ffill()

    n['done'] = n.apply(lambda x: corrective_date(x.done, x.date), axis=1)
    n['broken'] = n.apply(lambda x: fix_broken_time(x['broken'], x['done']), axis=1)
    _.append(n)

jam_breakdown = pd.concat(_)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 424/424 [00:16<00:00, 25.71it/s]


In [51]:
def fix_done_lewat_hari(done, day_end):
    date = str(day_end)
    done1 = str(done)

    con = pd.to_datetime(date) < pd.to_datetime(done1)

    if con == True:
        msg = day_end
    else:
        msg = done
    return msg

jam_breakdown['done'] = jam_breakdown.apply(lambda x: fix_done_lewat_hari(x['done'], x['actual_end_date']), axis=1)

In [52]:
# jam_breakdown = jam_breakdown[jam_breakdown.done!='nan']
jam_breakdown['broken'] = jam_breakdown['broken'].apply(lambda x: str(x))
jam_breakdown['done'] = jam_breakdown['done'].apply(lambda x: str(x))

In [53]:
jam_breakdown['broken'] = jam_breakdown.apply(lambda x: x.actual_end_date if pd.to_datetime(x.broken) > pd.to_datetime(x.done) else x.broken, axis=1)

In [54]:
# jam_breakdown = jam_breakdown[jam_breakdown.done!='nan']
jam_breakdown['broken'] = jam_breakdown['broken'].apply(lambda x: str(x))
jam_breakdown['done'] = jam_breakdown['done'].apply(lambda x: str(x))

In [55]:
_ = []
for i in tqdm(jam_breakdown.name.unique()):
    n = jam_breakdown[jam_breakdown.name==i]
    for j in n.date.unique():
        try:
            m = n[n.date==j]
    
            hm_maintenance = calculate_total_breakdown_without_double(m.broken.tolist(), m.done.tolist())
    
            stage = ';'.join(m.stage.unique().tolist())
            spk = ';'.join(m.spk_workshop.unique().tolist())
            req_type = ';'.join(m.requirement_type.unique().tolist())
            broken = ';'.join(m.broken.unique().tolist())
            done = ';'.join(m.done.unique().tolist())
    
            msg = m.head(1)
            msg['stage'] = stage
            msg['spk_workshop'] = spk
            msg['requirement_type'] = req_type
            msg['broken'] = broken
            msg['done'] = done
            msg['hm_maintenance (minute)'] = hm_maintenance
            _.append(msg)
        except:
            msg = m.head(1)
            msg['hm_maintenance (minute)'] = 0
            _.append(msg)

jam_breakdown = pd.concat(_)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 424/424 [01:09<00:00,  6.13it/s]


In [56]:
# _ = []
# for i in jam_breakdown.name.unique():
#     n = jam_breakdown[jam_breakdown.name==i]
#     n = n.merge(date[['date']], on='date', how='outer')

#     cat_eq = n.category_equipment.unique().tolist()[0]
#     eq_name = n.equipment_name.unique().tolist()[0]
#     eq_code = n.equipment_code.unique().tolist()[0]

#     n['category_equipment'].fillna(cat_eq, inplace=True)
#     n['equipment_name'].fillna(eq_name, inplace=True)
#     n['equipment_code'].fillna(eq_code, inplace=True)
#     n['name'].fillna(i, inplace=True)
#     n['stage'].fillna('-', inplace=True)
#     n['spk_workshop'].fillna('-', inplace=True)
#     n['requirement_type'].fillna('-', inplace=True)
#     n['hm_maintenance (minute)'].fillna(0, inplace=True)
    
#     n = n.sort_values('date', ascending=False).reset_index(drop=True)
#     _.append(n)

# jam_breakdown = pd.concat(_)
# jam_breakdown.to_excel('./Report/rekap breakdown equipment.xlsx', index=False)

In [57]:
def get_id(jam, equipment):
    jam = str(jam)
    eq = str(equipment)
    msg = '#' + jam + '#' + eq
    return msg

jam_operasional['id'] = jam_operasional.apply(lambda x: get_id(x['date'], x['name']), axis=1)
jam_breakdown['id'] = jam_breakdown.apply(lambda x: get_id(x['date'], x['name']), axis=1)

In [58]:
t2 = pd.merge(jam_operasional[['id','company','spk_project','project/name','production','move_type','date_engine_start','date_engine_end','hm_engine_actual','produksi','retase']], jam_breakdown[['id','stage','spk_workshop','broken','done','hm_maintenance (minute)']], how='outer', on='id')

t2['date'] = t2['id'].apply(lambda x: str(x).split('#')[1])
t2['name'] = t2['id'].apply(lambda x: str(x).split('#')[-1])
t2 = t2.merge(tbl_eq[['category_equipment','equipment_name','name']].drop_duplicates(), on='name', how='left')

In [59]:
t2['day'] = pd.to_datetime(t2.date).dt.day
t2['month'] = pd.to_datetime(t2.date).dt.month
t2['year'] = pd.to_datetime(t2.date).dt.year
t2 = t2.sort_values(['name','year','month','day'], ascending=False)
t2.reset_index(drop=True, inplace=True)
t2.drop(columns=['day','month','year'], inplace=True)
t2['hm_engine_actual'].fillna(0, inplace=True)
t2 = t2.rename(columns={'hm_engine_actual':'jam_produktivitas','hm_maintenance (minute)':'jam_breakdown'})
t2['jam_breakdown'].fillna(0, inplace=True)
t2['jam_breakdown'] = t2['jam_breakdown']/60

In [60]:
def get_jam_stanby_true(x):
    if x < 0:
        msg = 0
    else:
        msg = x
    return msg

t2['jam_standby'] = t2.apply(lambda x: get_jam_stanby_true(10 - (x.jam_produktivitas + x.jam_breakdown)), axis=1)
t2['move_type'].fillna('-', inplace=True)
t2['spk_workshop'].fillna('-', inplace=True)
t2['spk_project'].fillna('-', inplace=True)
t2['project/name'].fillna('-', inplace=True)
t2['company'].fillna('-', inplace=True)
t2['project/name'] = t2['project/name'].apply(lambda x: str(x))
t2['produksi'].fillna(0, inplace=True)
t2['retase'].fillna(0, inplace=True)
t2['production'].fillna('-', inplace=True)
t2['stage'].fillna('-', inplace=True)
t2['category_equipment'].fillna('Tidak Diketahui', inplace=True)
t2['equipment_name'].fillna('Tidak Diketahui', inplace=True)
t2['name'].fillna('Tidak Diketahui', inplace=True)
t2.drop(columns='id', inplace=True)
t2 = t2.drop_duplicates()

In [61]:
t2 = t2[['company','spk_project','spk_workshop','project/name','production','move_type','stage','category_equipment','equipment_name',
    'name','produksi','retase','date','date_engine_start','date_engine_end','broken','done','jam_produktivitas','jam_breakdown','jam_standby']]

In [62]:
def fix_kpi(ts, work, bd, stand):
    if ts == '-':
        stand = 10 - bd
        bd = bd
        work = 0
    else:
        if work == 0:
            work = 10 - bd
            bd = bd
            stand = 0
        else:
            work = work - bd
            stand = 0
            bd = bd
    msg = [work, bd, stand]
    return msg

In [63]:
t2['jam_produktivitas'] = t2.apply(lambda x: fix_kpi(x.spk_project, x.jam_produktivitas, x.jam_breakdown, x.jam_standby)[0], axis=1)
t2['jam_breakdown'] = t2.apply(lambda x: fix_kpi(x.spk_project, x.jam_produktivitas, x.jam_breakdown, x.jam_standby)[1], axis=1)
t2['jam_standby'] = t2.apply(lambda x: fix_kpi(x.spk_project, x.jam_produktivitas, x.jam_breakdown, x.jam_standby)[2], axis=1)

In [64]:
kpi = []
for i in tqdm(t2.name.unique().tolist()):
    n = t2[t2.name==i]

    cat_eq = n.category_equipment.unique().tolist()[0]
    eq_name = n.equipment_name.unique().tolist()[0]
    name = n.name.unique().tolist()[0]
    
    n = date.merge(n, on='date', how='left')
    n['company'].fillna('-', inplace=True)
    n['project/name'].fillna('-', inplace=True)
    n['spk_workshop'].fillna('-', inplace=True)
    n['spk_project'].fillna('-', inplace=True)
    n['move_type'].fillna('-', inplace=True)
    n['stage'].fillna('-', inplace=True)
    n['production'].fillna('-', inplace=True)
    n['category_equipment'].fillna(cat_eq, inplace=True)
    n['equipment_name'].fillna(eq_name, inplace=True)
    n['name'].fillna(name, inplace=True)
    n['jam_produktivitas'].fillna(0, inplace=True)
    n['jam_breakdown'].fillna(0, inplace=True)
    n['jam_standby'].fillna(0, inplace=True)
    n['produksi'].fillna(0, inplace=True)
    n['retase'].fillna(0, inplace=True)
    kpi.append(n)

# gabungkan data
kpi = pd.concat(kpi)
kpi['category_equipment'].replace('TidakDiketahui', 'Tidak Diketahui', inplace=True)
kpi.drop_duplicates(inplace=True)

# merge dengan data standar jarak tempuh move type
kpi = kpi.merge(tbl_jarak, on='move_type', how='left').rename(columns={'jarak':'standar_jarak'})
kpi['standar_jarak'].fillna(0, inplace=True)

# merge dengan data standar maximal muatan dt
kpi = kpi.merge(tbl_kapasitas[['equipment_name','kapasitas_maximal']], on='equipment_name', how='left')
kpi['kapasitas_maximal'].fillna(0, inplace=True)

# re-columns data kpi
kpi = kpi[['date','company','spk_project','project/name','production','move_type','spk_workshop','stage','category_equipment'
           ,'equipment_name','name','standar_jarak','jam_produktivitas','jam_breakdown','jam_standby','kapasitas_maximal','produksi','retase']]

kpi['total_jarak_tempuh'] = kpi.retase * kpi.standar_jarak
kpi.fillna(0, inplace=True)
kpi = kpi[kpi.equipment_name!='NISSAN CWA 260 X']

kpi = kpi.merge(tbl_waktu, on='move_type', how='left')
kpi['waktu'].fillna(0, inplace=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 432/432 [00:03<00:00, 112.65it/s]


In [65]:
# def retext_jam_produktivitas(work, produksi, retase, waktu):
#     if work == 0 and produksi != 0:
#         msg = retase * waktu
#     else:
#         msg = work
#     return msg

# list_index = kpi[(kpi.jam_produktivitas == 0) & (kpi.produksi!=0)].index.tolist()

In [66]:
# kpi['jam_produktivitas'] = kpi.apply(lambda x: retext_jam_produktivitas(x.jam_produktivitas, x.produksi, x.retase, x.waktu), axis=1)

# def repair_jam(i, work, bd, s):
#     if i in list_index:
#         # penyesuaian jam work
#         if work > 10:
#             work = 10 - bd
#             if work < 0:
#                 work = 0
#         else:
#             work = work
#             if work < 0:
#                 work = 0

#         # penyesuaian jam standby
#         s = 10 - (work + bd)
#         msg = [work, bd, s]
#     else:
#         msg = [work, bd, s]
#     return msg

# kpi['jam_produktivitas'] = kpi.reset_index().apply(lambda x: repair_jam(x['index'], x.jam_produktivitas, x.jam_breakdown, x.jam_standby)[0], axis=1)
# kpi['jam_breakdown'] = kpi.reset_index().apply(lambda x: repair_jam(x['index'], x.jam_produktivitas, x.jam_breakdown, x.jam_standby)[1], axis=1)
# kpi['jam_standby'] = kpi.reset_index().apply(lambda x: repair_jam(x['index'], x.jam_produktivitas, x.jam_breakdown, x.jam_standby)[2], axis=1)

# def get_project_status(x, bd, standby):
#     if standby >= 10:
#         msg = 'Standby'
#     elif bd > 0:
#         msg = 'Breakdown'
#     else:
#         msg = x
#     return msg

# kpi['project/name'] = kpi.apply(lambda x: get_project_status(x['project/name'], x['jam_breakdown'], x['jam_standby']), axis=1)

In [67]:
# kpi[kpi.jam_breakdown>=10].sort_values('name', ascending=False).name.unique().tolist()

## Task 1.2 : DASHBOARD

In [68]:
# buat fungsi untuk mengklasifikasi jenis ketersediaan unit
def get_jml_ketersediaan_unit(produktivitas, breakdown, standby):
    if produktivitas > 0:
        msg = 'In Work'
    elif breakdown > standby:
        msg = 'Broken'
    elif standby > breakdown:
        msg = 'Standby'
    else:
        msg = 'Broken'
    return msg

# get and apply function
t1_2 = kpi[['date','project/name','category_equipment','equipment_name','name','jam_produktivitas','jam_breakdown','jam_standby','produksi','retase','total_jarak_tempuh']]
t1_2['status'] = t1_2.apply(lambda x: get_jml_ketersediaan_unit(x.jam_produktivitas, x.jam_breakdown, x.jam_standby), axis=1)
t1_2['number'] = 1
t1_2 = t1_2[t1_2['project/name'].isin(['-','Mining KBM','Mining BDM','Rental BDM'])]
t1_2 = t1_2.groupby(['date','project/name','category_equipment','equipment_name','status'])[['number','produksi','retase']].sum().reset_index().rename(columns={'number':'count_unit'})

# read data detail informasi equipment
url_depresiasi = "https://docs.google.com/spreadsheets/d/1cjcBMY29etIAeQAwRDzlbpd4wExYkpvxk7s4tgVUD-w/export?format.xlsx"
try:
    os.remove('detail_depresiasi.xlsx')
except:
    None
    
output_filename = 'detail_depresiasi.xlsx'
# get the data from spreadsheet
response = requests.get(url_depresiasi)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)

# read data
dep = pd.read_excel(output_filename)
dep.columns = ['_'.join(i.lower().split()) for i in dep.columns]
dep.fillna(0, inplace=True)
dep.columns = ['company','category_equipment','equipment_name','name','date_pay','eq_age','dpp','lama_penyusutan',
               'price_aftar_salvage','depresiasi','insurance','interest','tax',]
eq = dep[['company','category_equipment','equipment_name','name']].drop_duplicates()
eq = eq.groupby(['category_equipment','equipment_name'])[['name']].count().reset_index().rename(columns={'name':'count_all_unit'})

# merge data
t1_2 = t1_2.merge(eq[['equipment_name','count_all_unit']], on='equipment_name', how='left').fillna(0).drop_duplicates()
t1_2 = t1_2.merge(tbl_kapasitas[['equipment_name','kapasitas_maximal']], on='equipment_name', how='left').fillna(0)

In [69]:
t1_2.to_excel('./Report/task1_2.xlsx', index=False)

## Task 2.2 : DASHBOARD

KPI Rekap

In [70]:
jam_operasional = df2.groupby(['date','name','company','project','production','move_type','equipment'])[['hm_engine_start','hm_engine_end','component_line/out_weight','component_line/in_weight']].sum().reset_index()
jam_operasional['produksi'] = (jam_operasional['component_line/in_weight'] - jam_operasional['component_line/out_weight'])/1000
jam_operasional.drop(columns=['component_line/out_weight','component_line/in_weight'], inplace=True)
jam_operasional['equipment'] = jam_operasional.equipment.apply(lambda x: str(x).split('/')[0])
jam_operasional = jam_operasional.rename(columns={'name':'ts','equipment':'name'})
jam_operasional = jam_operasional.merge(kpi[['category_equipment','equipment_name','name']].drop_duplicates(), on='name')
jam_operasional = jam_operasional[['date','ts','company','project','production','move_type','category_equipment',
                                   'equipment_name','name','hm_engine_start','hm_engine_end','produksi']]
jam_operasional['date'] = pd.to_datetime(jam_operasional.date).dt.date

# manipulate data agar sesuai kebutuhan
_ = []
for i in tqdm(jam_operasional.date.unique().tolist()):
    n = jam_operasional[jam_operasional.date == i]
    for j in n.name.unique().tolist():
        m = n[n.name==j]
        m = m[m.hm_engine_end!=0]
        count_ret = len(m)
        hm = m.head(1).hm_engine_end - m.head(1).hm_engine_start
        msg = m.head(1)
        msg['hm'] = hm
        msg['retase'] = count_ret
        _.append(msg)
jam_operasional = pd.concat(_)
jam_operasional.drop(columns='ts', inplace=True)
jam_operasional.columns = ['date','company','project/name','production','move_type','category_equipment','equipment_name','name','hm_engine_start','hm_engine_end','produksi','hm_engine_in_work_productivity','retase']
t2_1 = jam_operasional.copy()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 545/545 [00:30<00:00, 17.67it/s]


In [71]:
t2_1 = t2_1.groupby(['date','company','project/name','production','move_type','category_equipment','equipment_name','name'])[['hm_engine_in_work_productivity','produksi','retase']].sum().reset_index()

In [72]:
t2_1.to_excel('./Report/task2_1.xlsx', index=False)

## Task 3 : DASHBOARD

Buat perbandingan antara Biaya terhadap hasil produksi

In [73]:
t3 = df2.copy()
t3 = t3[t3.production=='Hauling Ore']
target = ['HINO ZS','HINO ZY','HONGYAN KINKAN430']
equipment = df[df.equipment_name.isin(target)==True][['category_equipment','equipment_name','equipment_code']].drop_duplicates()
equipment['equipment_detail'] = equipment.equipment_code.apply(lambda x: str(x).split('/')[0])
t3 = t3.merge(equipment, left_on='equipment', right_on='equipment_code', how='left')
t3 = t3[t3.equipment_name.isin(target)]
t3['produksi'] = t3['component_line/in_weight'] - t3['component_line/out_weight']

t3 = t3[['project/name','project','name','date','move_type','hm_engine_actual','production','employee','category_equipment','equipment_name','equipment_code','equipment_detail','component_line/in_weight','component_line/out_weight','produksi','state']]
t3.columns = [i.split('/')[-1] for i in t3.columns]

# group data produksi harian 
t3 = t3.groupby(['date','project','move_type','equipment_name','equipment_detail'])[['in_weight','out_weight','produksi']].sum().reset_index()
t3['date'] = t3['date'].dt.date
t3['date'] = t3['date'].astype(str)

# klasifikasi status dari tujuan produksi berdasarkan moving type
def get_produksi_status(x):
    x = str(x)
    con = x.split('- ')[-1]
    if con == 'KM 7':
        msg = 'Deposit'
    else:
        msg = 'Sell'
    return msg

t3['produksi_status'] = t3['move_type'].apply(lambda x: get_produksi_status(x))

# group data biaya harian
cost_t1 = t1.copy()
cost_t1 = cost_t1[['broken','equipment_name','name','cost']]
cost_t1['broken'] = cost_t1['broken'].dt.date
cost_t1 = cost_t1.groupby(['broken','equipment_name','name'])[['cost']].sum().reset_index()
cost_t1.columns = ['date','equipment_name','equipment_detail','cost']
cost_t1['date'] = cost_t1['date'].astype(str)
cost_t1 = cost_t1[cost_t1.equipment_name.isin(t3.equipment_name.unique().tolist())]

# merge kedua data
t3['id'] = t3.date +'#'+ t3.equipment_name +'#'+ t3.equipment_detail
cost_t1['id'] = cost_t1.date +'#'+ cost_t1.equipment_name +'#'+ cost_t1.equipment_detail
t3 = t3.merge(cost_t1[['id','cost']], on='id', how='outer')
t3 = t3[['id','project','move_type','produksi_status','in_weight','out_weight','produksi','cost']]
t3['date'] = t3['id'].apply(lambda x: str(x).split('#')[0])
t3['equipment_name'] = t3['id'].apply(lambda x: str(x).split('#')[1])
t3['equipment_detail'] = t3['id'].apply(lambda x: str(x).split('#')[-1])
t3.drop(columns='id', inplace=True)

t3 = t3[['date','project','move_type','produksi_status','equipment_name','equipment_detail','in_weight','out_weight','produksi','cost']]

# cleaning data
t3['cost'].fillna(0, inplace=True)
t3['in_weight'].fillna(0, inplace=True)
t3['out_weight'].fillna(0, inplace=True)
t3['produksi'].fillna(0, inplace=True)
t3['project'].fillna('Breakdown', inplace=True)
t3['move_type'].fillna('Breakdown', inplace=True)
t3['produksi_status'].fillna('Breakdown', inplace=True)

t3_bd = t3[t3.project=='Breakdown']
t3_bd = t3_bd[t3_bd.cost!=0]
t3_ac = t3[t3.project!='Breakdown']
t3 = pd.concat([t3_bd, t3_ac])
t3 = t3.sort_values('date', ascending=False).reset_index(drop=True)
t3['produksi'] = t3['produksi']/1000
t3.to_excel('./Report/task3.xlsx', index=False)

## Task 5 : DASHBOARD
Kumpulkan trend data produksi dan bandingkan dengan maximal muatan nya 

In [74]:
c2 = kpi.copy()
c2 = c2[c2['project/name']!='-']
c2 = c2[c2.retase>0]
c2['count_sheet'] = c2['production'].apply(lambda x: len(str(x).split(';')))
c2['target_produksi'] = c2.count_sheet * c2.kapasitas_maximal
c2 = c2[c2.target_produksi!=0]
c2 = c2[['date','project/name','production','move_type','spk_project','category_equipment','equipment_name','name','standar_jarak',
         'jam_produktivitas','jam_breakdown','jam_standby','kapasitas_maximal','produksi','target_produksi','retase','total_jarak_tempuh','count_sheet']]

def get_different(retase, c_spk_kerja, produksi, target_produksi):
    retase_round = np.round(retase)
    
    con = retase_round == c_spk_kerja
    diff = c_spk_kerja - retase
    p_diff = diff / c_spk_kerja

    selisih_target = target_produksi - produksi
    p_selisih_target = selisih_target/target_produksi

    msg = {
        'condition':[con],
        'diff':[diff],
        'p_diff':[p_diff],
        'selisih_target_produksi':[selisih_target],
        'p_selisih_target_produksi':[p_selisih_target]
    }

    return msg

ret = c2.iloc[0]['retase']
count_sheet = c2.iloc[0]['count_sheet']
produksi = c2.iloc[0]['produksi']
target_produksi = c2.iloc[0]['target_produksi']

c2['retase_berbeda'] = c2.apply(lambda x: get_different(x.retase, x.count_sheet, x.produksi, x.target_produksi)['condition'][0], axis=1)
c2['selisih_retase_berbeda'] = c2.apply(lambda x: get_different(x.retase, x.count_sheet, x.produksi, x.target_produksi)['diff'][0], axis=1)
c2['p_selisih_retase_berbeda'] = c2.apply(lambda x: get_different(x.retase, x.count_sheet, x.produksi, x.target_produksi)['p_diff'][0], axis=1)
c2['selisih_target_produksi'] = c2.apply(lambda x: get_different(x.retase, x.count_sheet, x.produksi, x.target_produksi)['selisih_target_produksi'][0], axis=1)
c2['p_selisih_target_produksi'] = c2.apply(lambda x: get_different(x.retase, x.count_sheet, x.produksi, x.target_produksi)['p_selisih_target_produksi'][0], axis=1)

# eda1 akan menunjukan grafik perbedaan muatan yang diproduksi dengan yang sebenarnya dapat tercapai.
eda1 = c2[['date','spk_project','project/name','production','move_type','equipment_name','name',
           'kapasitas_maximal','produksi','retase','count_sheet']].rename(columns={'kapasitas_maximal':'kapasitas'})

eda1['retase'] = eda1['retase'].apply(lambda x: int(np.round(x)))

# retase didapatkan dengan cara membagi hasil produksi dengan kapasitas DT
# apabila kapasitas DT di maksimalkan maka semestinya akan diperoleh hasil produksi yang maksimal kita nama produksi maksimal
eda1['produksi_max_retase'] = eda1.retase * eda1.kapasitas

# count_sheet merupakan jumlah sheet yang dibuat oleh operator. Umum nya jumlah sheet ini mewakili banyak retase yang dilakukan. Selanjutnya kita akan hitung berapa hasil maksimal dari banyak sheet apabila menggunakan maksimal muatas yang kita namai sebagai produksi max count sheet
eda1['produksi_max_count_sheet'] = eda1.count_sheet * eda1.kapasitas

eda1 = eda1.groupby(['date','spk_project','project/name','production','move_type','equipment_name','name'])[['kapasitas','produksi','retase','count_sheet','produksi_max_retase','produksi_max_count_sheet']].sum().reset_index()

eda1.to_excel('./Report/task5_2.xlsx', index=False)

In [75]:
cols = ['produksi','retase','count_sheet','produksi_max_retase','produksi_max_count_sheet']

eda2_ = []
for i in tqdm(cols):
    data = eda1.copy()
    values = data[i].tolist()
    data['status'] = i
    data = data[['date','spk_project','move_type','equipment_name','name','kapasitas','status']]
    data['value'] = values
    eda2_.append(data)

eda2 = pd.concat(eda2_)
eda2.to_excel('./Report/task5_1.xlsx', index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 87.30it/s]


In [76]:
check = df2.drop_duplicates()
check = check[check.production.isin(['Hauling Ore'])]
check['my'] = pd.to_datetime(check.date).dt.strftime('%m/%y')
check = check[check.my.isin(['01/24','02/24','03/24','04/24','05/24','06/24'])]
check = check.rename(columns={'name':'spk_project'})
check['name'] = check.equipment.apply(lambda x: str(x).split('/')[0])
check = check.merge(tbl_eq[['equipment_name','name']], on='name', how='left')
check['produksi'] = check['component_line/net_weight/voll'] / 1000

check = check.groupby(['date','spk_project','equipment_name','name'])[['produksi']].sum().reset_index()
check.groupby(['equipment_name'])['produksi'].agg([np.mean, np.max, np.min])

,mean,amax,amin
equipment_name,,,
HINO ZS,37.882666,47.69,26.38
HINO ZY,49.038162,76.74,24.48
HONGYAN KINKAN430,56.126976,72.28,37.90


In [77]:
check = check.merge(tbl_kapasitas[['equipment_name','kapasitas_maximal']], on='equipment_name', how='left')
check['kapasitas_maximal'] = check['kapasitas_maximal'] + 2
check[check.produksi > check.kapasitas_maximal].to_excel('check kebenaran data produksi 2024.xlsx', index=False)

## Task 6 : ANALISA

## Task 7 : DASHBOARD
- Dashboard Monitoring Prodical Maintenance Equipment
- Rerata Equipment harus masuk dalam periodical Maintenance

In [78]:
# # kumpul data historical dari setiap alat
# bd7 = df[['spk','broken','category_equipment','equipment_name','equipment_code','hm']]
# ts7 = df2[['name','date','equipment','hm_engine_start','hm_engine_end','actual_engine']]

# # transform data
# bd7['name'] = bd7['equipment_code'].apply(lambda x: str(x).split('/')[0])
# bd7 = bd7.rename(columns={'spk':'spk','broken':'date','hm':'hm_engine_end'})
# bd7 = bd7.drop(columns=['equipment_code'])
# bd7['status'] = 'b/d'
# bd7 = bd7[['status','spk','date','category_equipment','equipment_name','name','hm_engine_end']]

# ts7 = ts7.rename(columns={'name':'spk','equipment':'name'})
# ts7['name'] = ts7['name'].apply(lambda x: str(x).split('/')[0])

# # merge data
# ts7 = ts7.merge(bd7[['category_equipment','equipment_name','name']].drop_duplicates(), on='name', how='left')
# ts7['status'] = 'timesheet'
# ts7 = ts7[['status','spk','date','category_equipment','equipment_name','name','hm_engine_end']]

# # concat all data time sheet and data breadown
# t7 = pd.concat([ts7,bd7])
# t7['date'] = pd.to_datetime(t7.date).dt.date
# # t7 = t7[pd.to_datetime(t7.date).dt.year>2023]

# # kita asumsikan data tidak terdapat human error
# t7 = t7[t7.hm_engine_end!=0]

# t7 = t7.sort_values(['name','date','hm_engine_end'], ascending=True)
# t7 = t7.drop_duplicates(subset=['date','category_equipment','equipment_name','name','hm_engine_end'])

In [79]:
# # sample data untuk uji program

# list_unit_hino_zy = t7[t7.equipment_name=='HINO ZY'].name.unique().tolist()
# sample = t7[t7.name==list_unit_hino_zy[20]]

# # cleaning data
# sample['pct_change'] = sample.hm_engine_end.pct_change()
# sample = sample[sample['pct_change']>0]
# sample = sample[sample['pct_change']<sample['pct_change'].median()]
# sample = sample[pd.to_datetime(sample.date).dt.year>=2024]
# sample.drop(columns='pct_change', inplace=True)

# def get_increase_hm(list_hm):
#     msg = []
#     for i in range(len(list_hm)):
#         try:
#             value = list_hm[i + 1] - list_hm[i]
#             msg.append(value)
#         except:
#             msg.append(None)
#     msg = [0] + msg[:-1]   
#     return msg

# # apply function
# sample['increase_hm'] = get_increase_hm(sample.hm_engine_end.tolist())

In [80]:
# def cek_periode_ganti(hm_engine_end, eq_name):
#     base = tbl_periodical[tbl_periodical.equipment_name==eq_name].reset_index(drop=True)
#     base['params'] = [0 for i in range(len(base))]
    
#     increase_hm = get_increase_hm(hm_engine_end)
#     msg = []
#     for i in increase_hm:
#         # buat perulangan untuk menambahkan params
#         for k in range(len(base)):
#             base['params'][k] = base['params'][k] + i
            
#         base['con'] = base.params >= base.standar_hm
#         if True in base['con'].tolist():
#             n = base[base.con == True]
#             index_list = n.index.tolist()
#             for j in index_list:
#                 base['params'][j] = 0
#             msg.append(','.join(n.description.tolist()))
#         else:
#             msg.append(np.NaN)

#     return msg

In [81]:
# t7_ = []
# for i in tqdm(t7.name.unique().tolist()):
#     n = t7[t7.name==i]
#     # n = n[pd.to_datetime(n.date).dt.year>=2024]
    
#     hm_list = n['hm_engine_end'].tolist()
#     eq_name = n['equipment_name'].unique().tolist()[0]
    
#     n['periodical_maintenance'] = cek_periode_ganti(hm_list, eq_name)
#     t7_.append(n)
# t7 = pd.concat(t7_)

In [82]:
# check = t7[t7.name=='D65-10'].periodical_maintenance.value_counts().reset_index().to_dict()

In [83]:
# # Ubah kolom 'index' menjadi unik
# unique_index = []
# for index_list in check['index'].values():
#     unique_index.extend(index_list.split(','))

# unique_index = sorted(set(unique_index))

# # Buat DataFrame baru dengan indeks unik
# new_data = {
#     'index': [', '.join(unique_index)] * len(check['index']),
#     'periodical_maintenance': check['periodical_maintenance'].values()
# }

In [84]:
# check = t1[pd.to_datetime(t1.done).dt.strftime('%m/%y').isin(['02/23'])]

In [85]:
# check[check.requirement_type=='External'].to_excel('check_external.xlsx', index=False)

## Task 8 : ANALISA

## Task 9 : ANALISA

## Task 10 : DASHBOARD

In [86]:
company = kpi[['date','company','name']].drop_duplicates()
company['id'] = company.apply(lambda x: get_id(x['date'], x['name']), axis=1)

t10 = t3.copy()
t10['id'] = t10.apply(lambda x: get_id(x['date'], x['equipment_detail']), axis=1)
t10 = t10.merge(company[['company','id']], on='id', how='left')

# KONTRAK KBM : MINING KBM
# KONTRAK BLOK 8 : MINIM BDM
# RENTAL BLOK 8 : [RENTAL BDM] [BLOK 5 - PABRIK]
# RENTAL FEEDING KM 7 : [RENTAL BDM] [KM7 - PABRIK]

# def get_kontrak(project, move):
#     if project == 'Mining KBM':
#         msg = 'Kontrak KBM'
        
#     elif project == 'Mining BDM':
#         msg = 'Kontrak Blok 8'
        
#     elif project == 'Rental BDM':
#         if move == 'BLOK 5 - PABRIK':
#             msg = 'Rental Blok 8'
            
#         elif move == 'KM7 - PABRIK':
#             msg = 'Feeding KM7'
            
#         else:
#             msg = 'Tidak Diketahui'

#     else:
#         msg = 'Tidak Diketahui'
#     return msg

# # APPLY FUNCTION
# t10['kontrak'] = t10.apply(lambda x: get_kontrak(x.project, x.move_type), axis=1)
t10 = t10.rename(columns={'project':'kontrak'})
t10 = t10[t10.kontrak!='Tidak Diketahui']
t10 = t10[['date','company','kontrak','move_type','equipment_name','equipment_detail','in_weight','out_weight','produksi','cost']]

# mengisi tanggal kosong pada masing-masing kontrak
_ = []
for i in tqdm(t10.kontrak.unique().tolist()):
    n = t10[t10.kontrak == i]

    move = n.move_type.unique().tolist()[0]

    n = date.merge(n, on='date', how='left')
    n['company'].fillna('Tidak Diketahui', inplace=True)
    n['kontrak'].fillna(i, inplace=True)
    n['move_type'].fillna(move, inplace=True)
    n['equipment_name'].fillna('Tidak Diketahui', inplace=True)
    n['equipment_detail'].fillna('Tidak Diketahui', inplace=True)
    n['in_weight'].fillna(0, inplace=True)
    n['out_weight'].fillna(0, inplace=True)
    n['produksi'].fillna(0, inplace=True)
    n['cost'].fillna(0, inplace=True)
    _.append(n)

t10 = pd.concat(_)
t10 = t10.merge(tbl_kapasitas[['equipment_name','kapasitas_maximal']], on='equipment_name', how='left')

retase = kpi[kpi['spk_project']!='-']
retase = retase[['date','name','retase']]
retase['id'] = retase.date + '#' + retase.name

t10['id'] = t10.date + '#' + t10.equipment_detail
t10 = t10.merge(retase[['id','retase']], on='id', how='left').fillna(0).drop(columns='id')

t10['kapasitas_maximal'].fillna(0, inplace=True)
t10.to_excel('./Report/dashboard_request_pak_jon_1.xlsx', index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 68.95it/s]


## Task 11 : ANALISA

Analisa pemakaian bahan bakar

In [87]:
# date['month'] = pd.to_datetime(date.date).dt.strftime('%m/%y')

In [88]:
# # read and cleaning data
# t11 = pd.read_excel('/Users/dickyaryanto/Documents/PT Sentosa Abadi Mining/Data/data pemakaian fuel.xlsx')
# t11['name'] = t11['name'].apply(lambda x: ''.join(x.split()))
# t11['date'] = pd.to_datetime(t11.date).dt.strftime('%m/%y')
# t11.replace('-', 0, inplace=True)
# t11['hm'] = t11['hm'].astype(float)
# t11['produksi'] = t11['produksi'].astype(float)
# t11['retase'] = t11['retase'].astype(float)
# t11['konsumsi'] = t11['konsumsi'].astype(float)
# t11 = t11.merge(tbl_kapasitas[['equipment_name','kapasitas_maximal']], on='equipment_name', how='left').fillna(0).rename(columns={'kapasitas_maximal':'kap_max'})


# def get_hm(hm, produksi, kap_max, retase, month):
#     count_day = len(date[date.month==month])
#     actual_produksi = count_day * 4 * kap_max
#     try:
#         if hm != 0:
#             msg = hm
#         else:
#             f1 = (produksi / actual_produksi) * 10 * count_day
#             f2 = (retase / (count_day * 4)) * 10 * count_day
#             msg = (f1 + f2)/2
#     except:
#         msg = 0

#     return msg

# # apply function
# t11['hm'] = t11.apply(lambda x: get_hm(x.hm, x.produksi, x.kap_max, x.retase, x['date']), axis=1)

# # sns.pairplot(t11[t11.production=='Hauling Ore'][['hm','produksi','retase','konsumsi']])
# # plt.tight_layout()
# # plt.show()

In [89]:
# sns.pairplot(t11[t11.production!='Hauling Ore'][['hm','konsumsi']])
# plt.tight_layout()
# plt.show()

In [90]:
# t11[t11.production=='Hauling Ore'].drop(columns='kap_max').corr()

In [91]:
# t11[t11.production!='Hauling Ore'].drop(columns='kap_max')[['hm','konsumsi']].corr()

In [92]:
# ho = t11[t11.production=='Hauling Ore']
# nho = t11[t11.production!='Hauling Ore'].drop(columns=['retase','produksi','kap_max'])

In [93]:
# from sklearn.cluster import KMeans
# from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# scale = MinMaxScaler()
# le = LabelEncoder()

# xho = ho.copy()
# xnho = nho.copy()

# # transformasi tipe data kategorikal
# for i in xho.select_dtypes('object').columns.tolist():
#     xho[i] = le.fit_transform(xho[i])

# for i in xnho.select_dtypes('object').columns.tolist():
#     xnho[i] = le.fit_transform(xnho[i])

# # transformasi tipe data numerik
# for i in xho.select_dtypes('number').columns.tolist():
#     xho[i] = scale.fit_transform(xho[[i]])

# for i in xnho.select_dtypes('number').columns.tolist():
#     xnho[i] = scale.fit_transform(xnho[[i]])

In [94]:
# # buat evaluasi score inertia untuk XHO
# score_xho = []
# for i in range(2, 10):
#     kmeans = KMeans(n_clusters=i)
#     model = kmeans.fit(xho)
#     score_xho.append(model.inertia_)

# plt.title('Score Inertia Data Production Hauling Ore', fontsize=12, pad=15)
# plt.plot(range(2, 10), score_xho, marker='o')
# plt.tight_layout()
# plt.show()

# # print
# print('Data production Hauling Ore beter using 4 clusters')

# # define function using 4 clusters
# kmeans = KMeans(n_clusters=2)
# xho['c1'] = kmeans.fit_predict(xho)

In [95]:
# # buat evaluasi score inertia untuk XNHO
# score_xnho = []
# for i in range(2, 10):
#     kmeans = KMeans(n_clusters=i)
#     model = kmeans.fit(xnho)
#     score_xnho.append(model.inertia_)

# plt.title('Score Inertia Data Production != Hauling Ore', fontsize=12, pad=15)
# plt.plot(range(2, 10), score_xnho, marker='o')
# plt.tight_layout()
# plt.show()

# # print
# print('Data production != Hauling Ore beter using 3 clusters')

# # define function using 3 clusters
# kmeans = KMeans(n_clusters=2)
# xnho['c1'] = kmeans.fit_predict(xnho)

In [96]:
# # gabung data
# ho = pd.concat([ho, xho[['c1']]], axis=1)
# nho = pd.concat([nho, xnho[['c1']]], axis=1)

In [97]:
# sns.pairplot(ho[['hm','produksi','retase','konsumsi','c1']], hue='c1')

In [98]:
# corr_app = ho[ho.company=='CV. Adil Prima Perkasa'][['hm','retase','produksi','konsumsi']].corr()
# plt.figure(figsize=(8, 5))
# plt.title('Persentase Keterhubungan Antar Parameter', fontsize=19, pad=15)
# sns.heatmap(corr_app, linewidth=2, annot=True)

# plt.yticks(rotation=0)

# plt.tight_layout()
# # plt.savefig('./Images/Korelasi antar parameter app.svg', dpi=1000)

In [99]:
# corr_sa = ho[['hm','retase','produksi','konsumsi']].corr()
# plt.figure(figsize=(8, 5))
# plt.title('Persentase Keterhubungan Antar Parameter', fontsize=19, pad=15)
# sns.heatmap(corr_sa, linewidth=2, annot=True)

# plt.yticks(rotation=0)

# plt.tight_layout()
# # plt.savefig('./Images/Korelasi antar parameter.svg', dpi=1000)

In [100]:
# corr_sa = ho[ho.company=='CV. SENTOSA ABADI'][['hm','retase','produksi','konsumsi']].corr()
# plt.figure(figsize=(8, 5))
# plt.title('Persentase Keterhubungan Antar Parameter', fontsize=19, pad=15)
# sns.heatmap(corr_sa, linewidth=2, annot=True)

# plt.yticks(rotation=0)

# plt.tight_layout()
# # plt.savefig('./Images/Korelasi antar parameter sa.svg', dpi=1000)

In [101]:
# sns.pairplot(ho[['hm','produksi','retase','konsumsi','c1']], hue='c1')
# # plt.savefig('./Images/pair plot.svg', dpi=1000)

In [102]:
# ho['number'] = 1
# nho['number'] = 1

In [103]:
# eda1 = ho.groupby(['date','company'])[['produksi','konsumsi']].sum().reset_index()

In [104]:
# plt.figure(figsize=(9, 5))
# splot = sns.barplot(eda1.sort_values('company', ascending=False), x='date', y='produksi', hue='company')

# # Add annotations using splot
# for p in splot.patches:
#     try:
#         splot.annotate(rupiah(p.get_height()).split()[-1].split(',')[0], 
#                        (p.get_x() + p.get_width() / 2., p.get_height()), 
#                        ha = 'center', va = 'center', 
#                        xytext = (0, 10), 
#                        textcoords = 'offset points',
#                        fontsize=12)
#     except:
#         splot.annotate(p.get_height(), 
#                (p.get_x() + p.get_width() / 2., p.get_height()), 
#                ha = 'center', va = 'center', 
#                xytext = (0, 10), 
#                textcoords = 'offset points',
#                fontsize=12)
# plt.xlabel('')
# plt.yticks([])
# plt.ylabel('')
# plt.ylim(ymin=0, ymax=160000)
# plt.legend('')
# plt.tight_layout()
# plt.savefig('./Images/perbandingan produksi sa dan app.svg', dpi=1000)

In [105]:
# plt.figure(figsize=(9, 5))
# splot = sns.barplot(eda1.sort_values('company', ascending=False), x='date', y='konsumsi', hue='company')

# # Add annotations using splot
# for p in splot.patches:
#     try:
#         splot.annotate(rupiah(p.get_height()).split()[-1].split(',')[0], 
#                        (p.get_x() + p.get_width() / 2., p.get_height()), 
#                        ha = 'center', va = 'center', 
#                        xytext = (0, 10), 
#                        textcoords = 'offset points',
#                        fontsize=12)
#     except:
#         splot.annotate(p.get_height(), 
#                (p.get_x() + p.get_width() / 2., p.get_height()), 
#                ha = 'center', va = 'center', 
#                xytext = (0, 10), 
#                textcoords = 'offset points',
#                fontsize=12)

# plt.xlabel('')
# plt.yticks([])
# plt.ylabel('')
# plt.ylim(ymin=0, ymax=150000)
# plt.legend('')
# plt.tight_layout()
# plt.savefig('./Images/perbandingan konsumsi sa dan app.svg', dpi=1000)

In [106]:
# def pct(data):
#     _ = []
#     for i in range(len(data) - 1):
#         values = [data[i], data[i+1]]
#         msg = np.round(((np.max(values) - np.min(values)) / np.max(values))*100, 2)
#         _.append(msg)
#     return _

In [107]:
# print('1. PCT Produksi SA :',pct(eda1[eda1.company=='CV. SENTOSA ABADI'].produksi.tolist()))
# print('2. PCT Konsumsi SA :', pct(eda1[eda1.company=='CV. SENTOSA ABADI'].konsumsi.tolist()))

In [108]:
# print('1. PCT Produksi APP :',pct(eda1[eda1.company=='CV. Adil Prima Perkasa'].produksi.tolist()))
# print('2. PCT Konsumsi APP :', pct(eda1[eda1.company=='CV. Adil Prima Perkasa'].konsumsi.tolist()))

In [109]:
# eda1['konsumsi/produksi'] = eda1.konsumsi / eda1.produksi

In [110]:
# eda1.groupby(['company'])[['konsumsi/produksi']].mean()

In [111]:
# eda2 = kpi[kpi.company.isin(eda1.company.unique().tolist())]
# eda2['number'] = 1
# eda2['month'] = pd.to_datetime(eda2.date).dt.strftime('%m/%y')
# eda2['kontrak'] = eda2.apply(lambda x: get_kontrak(x['project/name'], x.move_type), axis=1)
# eda2 = eda2[eda2.month.isin(['01/24','02/24','03/24'])]
# eda2 = eda2[eda2.move_type.isin(['KBM - PABRIK','BLOK 8 - PABRIK'])]

In [112]:
# eda3 = eda2.groupby(['month','company','move_type'])[['produksi','retase','jam_produktivitas','number','total_jarak_tempuh']].sum().fillna(0).reset_index()

In [113]:
# eda4 = eda3[['month','company','total_jarak_tempuh','jam_produktivitas']]
# eda4 = eda4.drop(index=2)
# eda4['konsumsi'] = [125908, 93036, 92775, 126492, 84757]

In [114]:
# sns.barplot(eda4.sort_values('company', ascending=False), x='month', y='jam_produktivitas', hue='company')

In [115]:
# node3 = eda2.groupby(['company','month'])[['produksi']].sum().reset_index().rename(columns={'produksi':'produksi Node3'})
# node3 = node3.set_index(['month','company'])

# excel = eda1.groupby(['company','date'])[['produksi']].sum().reset_index().rename(columns={'produksi':'produksi MPE'})
# excel = excel[excel.date.isin(['02/24','03/24'])]
# excel = excel.rename(columns={'date':'month'})
# excel = excel.set_index(['month','company'])

# dif = pd.concat([node3, excel], axis=1).fillna(0)
# dif = dif.reset_index().sort_values('month', ascending=True)
# dif['Node3 - MPE'] = dif['produksi Node3'] - dif['produksi MPE']
# dif = dif.set_index(['month','company'])
# dif 

In [116]:
# lts = pd.read_excel('Learning CC bawahtanah.xlsx')
# lts = l[['PROCESS DATE',
#  'TYPE',
#  'MODEL',
#  'EQUIP NO',
#  'MATERIAL',
#  'SUMBER',
#  'TUJUAN',
#  'PROCESS',
#  'ACTIVITY',
#  'WHRS',
#  'PRODUKSI',
#  'CATEGORY',
#  'CLASS UNIT']]

## TASK 12 : DASHBOARD

buat trend biaya maintenance repair unit

In [117]:
# # read data planing hino
# plan_hino = pd.read_excel('/Users/dickyaryanto/Documents/PT Sentosa Abadi Mining/Data/estimasi plan biaya maintenance hino.xlsx')
# plan_hino['status'] = 'Plan'

# # read data detail informasi equipment
# url_depresiasi = "https://docs.google.com/spreadsheets/d/1cjcBMY29etIAeQAwRDzlbpd4wExYkpvxk7s4tgVUD-w/export?format.xlsx"

# try:
#     os.remove('detail_depresiasi.xlsx')
# except:
#     None
    
# output_filename = 'detail_depresiasi.xlsx'

# # get the data from spreadsheet
# response = requests.get(url_depresiasi)
# if response.status_code == 200:
#     with open(output_filename, "wb") as f:
#         f.write(response.content)
        
# eq_info = pd.read_excel('detail_depresiasi.xlsx')
# eq_info.columns = ['_'.join(i.lower().split()) for i in eq_info.columns]

## TASK 13 : ANALISA

Analisa kebutuhan meeting bersama **BOD** untuk evaluasi **Semester-1 2024** sebagai pertimbangan kebijakan **Planing Semester-2 2024**.

In [118]:
# data1 = pd.read_excel('./Report/task2.xlsx')
# data2 = pd.read_excel('./Report/cc dashboard 2.xlsx')

In [119]:
# # reduce parameter in data1
# data1 = data1[['date','project/name','production','move_type','category_equipment',
#                'equipment_name','name','produksi','retase','total_jarak_tempuh']]

# # buat kolom id untuk merge
# data1['id'] = data1.apply(lambda x: str(x.date) + '#' + x['name'], axis=1)
# data2['id'] = data2.apply(lambda x: str(x.date) + '#' + x['name'], axis=1)

In [120]:
# # merge data
# data = data2.drop(columns=['name','date','produksi']).merge(data1[['id','project/name','production','move_type','retase','produksi']], on='id', how='outer')
# data['date'] = data['id'].apply(lambda x: str(x).split('#')[0])
# data['name'] = data['id'].apply(lambda x: str(x).split('#')[-1])
# data.drop(columns='id', inplace=True)

# # repoisiton columns dataframe
# data = data[['date','project/name','production','move_type','category_equipment','equipment_name','hm',
#       'rm','fuel','tyre','depresiasi','insurance','interest','tax','ovh','retase','total_jarak_tempuh','produksi']]

# # data cleaning
# data['project/name'].fillna('-', inplace=True)
# data['production'].fillna('-', inplace=True)
# data['move_type'].fillna('-', inplace=True)
# data['ovh'].fillna(0, inplace=True)
# data['depresiasi'].fillna(0, inplace=True)
# data['insurance'].fillna(0, inplace=True)
# data['interest'].fillna(0, inplace=True)
# data['tax'].fillna(0, inplace=True)
# data['retase'].fillna(0, inplace=True)

# # data transforming
# data['my'] = pd.to_datetime(data.date).dt.strftime('%m/%y')
# data['month'] = pd.to_datetime(data.date).dt.month
# data['years'] = pd.to_datetime(data.date).dt.year
# data['cost_all'] = data.rm + data.fuel + data.tyre + data.depresiasi + data.insurance + data.tax + data.interest + data.ovh

In [121]:
# # buat fungsi untuk klasifikasi semester busines
# def get_semester(x):
#     if x < 7:
#         msg = 'Semester-1'
#     else:
#         msg = 'Semester-2'
#     return msg

# data['semester'] = data['month'].apply(get_semester)

### Task 13.1 : Evaluasi trend produksi/cost semester-1 2023 & 2024

In [122]:
# t13 = data[(data.semester=='Semester-1') & (data.production=='Hauling Ore')]
# t13 = t13.groupby(['semester','years','month','my'])[['rm','fuel','tyre','depresiasi','insurance',
#                                                       'interest','tax','ovh','hm','retase','total_jarak_tempuh','produksi','cost_all']].sum().reset_index()
# t13['cost/produksi'] = t13.cost_all / t13.produksi

In [123]:
# m = t13
# # m['my'] = pd.to_datetime(m.date).dt.strftime('%m/%y')
# # m['years'] = pd.to_datetime(m.date).dt.year
# m = m.groupby(['years','my'])[['fuel']].sum().reset_index()
# m[m.years == 2024]

In [124]:
# t13 = data[(data.semester=='Semester-1') & (data.production=='Hauling Ore')]
# t13 = t13[t13['project/name'].isin(['Mining KBM','Mining BDM'])].rename(columns={'project/name':'kontrak'})

In [125]:
# t13 = t13.groupby(['semester','years','month','my','kontrak'])[['rm','fuel','tyre','depresiasi','insurance','interest','tax','ovh','hm','retase','total_jarak_tempuh','produksi','cost_all']].sum().reset_index()

## TASK 14 : DASHBOARD LOGISTIC

dashboard untuk monitoring item keluar pada divisi logistik

In [126]:
t14 = df.copy()
t14 = t14[t14.picking_reference!=False]
t14['name'] = t14.equipment_code.apply(lambda x: str(x).split('/')[0])
t14 = t14.drop(columns=['equipment_code'])
t14 = t14[['company','spk','stage','picking_reference','picking_status','schedule','oprator','category_equipment','equipment_name',
           'name','requirement_type','type_maintenance','category_maintenance','description','qty','uom']].rename(columns={'schedule':'date'})
t14['description'] = t14['description'].apply(lambda x: str(x).split('] ')[-1])
t14['date'] = pd.to_datetime(t14['date']).dt.date
t14['category_maintenance'] = t14['category_maintenance'].apply(retext_category_maintenance)
t14 = t14[t14.qty!=0]

In [127]:
t14.to_excel('./Report/task14.xlsx', index=False)

## TASK 15 : ANALISA

Klasifikasi alasan kerusakan dan retase tidak tercapai

## TASK 16 : DASHBOARD CICO BOD

In [128]:
# # buat fungsi untuk klasifikasi jam keluar
# def klasifikasi_jam_keluar(x):
#     hour = x.hour
#     if hour < 7:
#         msg = 'Keluar Sebelum 07:00'
#     elif hour == 7:
#         msg = 'Keluar Tepat 07:00'
#     else:
#         msg = 'Keluar Setelah 07:00'
#     return msg

In [129]:
# # terapkan fungsi
# cico['status_keluar'] = cico.jam_keluar.apply(klasifikasi_jam_keluar)

# # buat fungsi untuk klasifikasi driver bekerja
# def klasifikasi_jam_kerja(x):
#     if x >= 10:
#         msg = 'Bekerja dalam 10 jam'
#     else:
#         msg = 'Bekerja dibawah 10 jam'
#     return msg

# # terapkan fungsi
# cico['status_jam_kerja'] = cico.total_jam_kerja.apply(klasifikasi_jam_kerja)
# cico['number'] = 1
# cico.to_excel('./Report/cico.xlsx', index=False)

## TASK 17 : ANALISA

Analisa cost repair and maintenance untuk beberapa dt

In [130]:
dt_analisa = pd.read_excel('dt analisa pak alfret.xlsx', names=['name'])
dt_analisa['name'] = dt_analisa.name.apply(lambda x: str(x).split()[1])
dt_analisa['name'] = dt_analisa['name'].apply(lambda x: '-'.join(str(x).split('-')[1:]))

In [131]:
t17 = t1[['broken','done','category_equipment','equipment_name','name','category_maintenance','description','qty','uom','cost']]
t17['date'] = pd.to_datetime(t17['done']).dt.date
t17['my'] = pd.to_datetime(t17['date']).dt.strftime('%m/%y')
t17 = t17[t17.my.isin(['01/24','02/24','03/24','04/24','05/24'])]
t17 = t17[['date','my','category_equipment','equipment_name','name','category_maintenance','description','qty','uom','cost']]

In [132]:
t17 = t17[t17.name.isin(dt_analisa.name.tolist())]

In [133]:
t17.to_excel('Analisa Cost Maintenance.xlsx', index=False)

## TASK 18 : ANALISA LAPORAN DRIVER DT

In [134]:
def try_transform_time(x):
    n = pd.DataFrame({'time':[x]})
    
    try:
        msg = pd.to_datetime(n['time']).dt.time
        msg = msg[0]
    except:
        msg = 'SPK Belum Input'

    return msg

# ambil tanggal hari kemarin
date_before = date['date'].tolist()[-2]

# filter data absen driver menggunakan tanggal kemarin
dt_rusak = absen_driver[absen_driver.date==date_before]
dt_rusak = dt_rusak[dt_rusak.keterangan!='TARGET']
dt_rusak = dt_rusak[['name','project/name','date','jam_keluar','jam_masuk','total_jam_kerja','retase','keterangan']]
dt_rusak['name'] = dt_rusak.name.apply(lambda x: 'DT-' + str(x).split('-')[-1])
dt_rusak['date'] = dt_rusak['date'].apply(lambda x: str(x).split()[0])

In [135]:
# detail conditional
unit_dt_rusak = ["DT-326","DT-346","DT-352","DT-357","DT-359","DT-360","DT-363","DT-369","DT-370","DT-371","DT-374"]

In [136]:
dt_rusak = dt_rusak[dt_rusak.name.isin(unit_dt_rusak)]

In [137]:
# filter data WS menggunakan tanggal kemarin
bd = df.copy()
bd['date'] = bd['broken'].apply(lambda x: str(x).split()[0])
bd = bd[bd['date']==date_before]
bd = bd.rename(columns={'spk':'spk_workshop'})

bd['name'] = bd.equipment_code.apply(lambda x: str(x).split('/')[0])

def get_hm_maintenance(broken, done):
    try:
        value = done - broken
    except:
        value = 0

    msg = timedelta_to_hour(value)
    if msg > 0:
        msg = msg
    else:
        msg = 0
    return msg
bd['hm_maintenance'] = bd.apply(lambda x: get_hm_maintenance(x.broken, x.done), axis=1)
bd['hm_maintenance'] = bd.apply(lambda x: 'On Progress' if x.stage != 'DONE' else x.hm_maintenance, axis=1)
bd['done'] = bd.apply(lambda x: 'On Progress' if x.stage != 'DONE' else x.done, axis=1)

# merge data
dt_rusak = dt_rusak.merge(bd[['name','stage','spk_workshop','requirement_type','broken','done','hm_maintenance']], on='name', how='inner').rename(columns={'project/name':'kontrak','total_jam_kerja':'jam_kerja'})

# dt_rusak['spk_workshop'].fillna('SPK Belum Input', inplace=True)
# dt_rusak['stage'].fillna('SPK Belum Input', inplace=True)
# dt_rusak['requirement_type'].fillna('SPK Belum Input', inplace=True)

# dt_rusak['broken'].fillna('SPK Belum Input', inplace=True)
# dt_rusak['broken'] = dt_rusak['broken'].apply(lambda x: str(x).split(';')[0])

# dt_rusak['done'].fillna('SPK Belum Input', inplace=True)
# dt_rusak['done'] = dt_rusak['done'].apply(lambda x: str(x).split(';')[-1])

dt_rusak['hm_maintenance'].fillna(0, inplace=True)

dt_rusak = dt_rusak[dt_rusak.requirement_type.apply(lambda x: 'External' in x.split(';'))!=True]

def get_time(list_time, status):
    try:
        if status == 'Broken':
            msg = pd.DataFrame({'time':list_time.split(';')})
            msg['time'] = pd.to_datetime(msg.time)
            msg = msg.sort_values('time', ascending=True)
            msg['time'] = msg['time'].apply(lambda x: str(x))
            msg = msg.head(1)['time'].unique().tolist()[0]
        else:
            msg = pd.DataFrame({'time':list_time.split(';')})
            msg['time'] = pd.to_datetime(msg.time)
            msg = msg.sort_values('time', ascending=False)
            msg['time'] = msg['time'].apply(lambda x: str(x))
            msg = msg.head(1)['time'].unique().tolist()[0]
    except:
        msg = 'SPK Belum Input'
    return msg
    
# dt_rusak = dt_rusak.groupby(['date','kontrak','name','requirement_type','spk_workshop','stage','jam_masuk',
#                              'jam_keluar','retase','broken','done','hm_maintenance','keterangan'])[['keterangan']].count()
    
# dt_rusak['broken'] = dt_rusak.apply(lambda x: get_time(x.broken, "Broken"), axis=1)
# dt_rusak['done'] = dt_rusak.apply(lambda x: get_time(x.done, "Done"), axis=1)
# dt_rusak['requirement_type'] = dt_rusak.requirement_type.apply(lambda x: 'Internal' if 'External' not in x.split(';') else 'Gabungan')
# dt_rusak  = dt_rusak.set_index(['date','kontrak','name','spk_workshop','requirement_type','stage'])[['jam_keluar','jam_masuk','jam_kerja','retase','broken','done','hm_maintenance','keterangan']]

# load data
# dt_rusak.to_excel('./Rekap kerusakan DT/Rekap dt ' + date_before + '.xlsx')

In [138]:
_ = []
for i in unit_dt_rusak:
    con = i in dt_rusak.name.unique().tolist()
    if con == False:
        _.append(i)

In [139]:
_

['DT-326',
 'DT-346',
 'DT-352',
 'DT-357',
 'DT-359',
 'DT-360',
 'DT-363',
 'DT-369',
 'DT-370',
 'DT-371',
 'DT-374']

In [140]:
c = df.copy()
c = c[['company','spk','stage','broken','done','category_maintenance','description','note']]
c = c[c.category_maintenance=='TYRE']
c = c[c.description!='Service']
c = c[pd.to_datetime(c.broken).dt.year==2024]
c = c[c.description.apply(lambda x: ' '.join(x.split('] ')[-1].split()[:2]) == 'BAN LUAR')]

In [141]:
c

,company,spk,stage,broken,done,category_maintenance,description,note
242,CV. SENTOSA ABADI,97071,DONE,2024-08-07 17:13:52,2024-08-07 17:50:00,TYRE,[BLAEOLUS-AGM-120024] BAN LUAR AEOLUS AGM 10 1...,<p>KET :</p><p>GANTI BAN BARU AEOLUS NO. 9 DAN...
244,CV. SENTOSA ABADI,97069,DONE,2024-08-07 15:05:49,2024-08-07 16:15:00,TYRE,[BLAEOLUS-AGM-120024] BAN LUAR AEOLUS AGM 10 1...,<p>KET :</p><p>GANTI BAN BARU AEOLUS + VELEG B...
266,CV. SENTOSA ABADI,97047,DONE,2024-08-07 13:51:25,2024-08-07 16:00:00,TYRE,[BLAEOLUS-AGM-120024] BAN LUAR AEOLUS AGM 10 1...,<p>KET :</p><p>GANTI BAN BARU AEOLUS NO. 5.6.7...
353,CV. SENTOSA ABADI,96962,DONE,2024-08-06 15:12:05,2024-08-06 15:30:00,TYRE,[BLAEOLUS-AGM-120024] BAN LUAR AEOLUS AGM 10 1...,"<p>KET :</p><p>1. GANTI BAN BARU AEOLUS NO. 1,..."
360,CV. SENTOSA ABADI,96956,DONE,2024-08-06 14:46:04,2024-08-06 15:26:00,TYRE,[BLAEOLUS-AGM-120024] BAN LUAR AEOLUS AGM 10 1...,<p>KET : </p><p>1. GANTI BAN SPER DI NO 7</p><...
...,...,...,...,...,...,...,...,...
61331,CV. SENTOSA ABADI,75540,DONE,2024-01-27 16:11:40,2024-01-27 17:10:00,TYRE,[BLCHA120024R] BAN LUAR CHAOYANG CB972E 1.200 R24,<p><b>KET :</b></p><p><b>1.GANTI BAN SPER 1SET...
61755,CV. SENTOSA ABADI,74023,DONE,2024-01-14 15:20:26,2024-01-14 15:45:00,TYRE,[BLAEOLUS-AGM-120024] BAN LUAR AEOLUS AGM 10 1...,<p><br></p>
61872,CV. SENTOSA ABADI,73503,DONE,2024-01-10 08:58:54,2024-01-11 11:00:00,TYRE,[BLAEOLUS-AGM-120024] BAN LUAR AEOLUS AGM 10 1...,<p>1. GANTI BAN DALAM BEKAS 1 PCS</p><p>2. VEL...
62058,CV. SENTOSA ABADI,72840,DONE,2024-01-03 10:12:00,2024-01-03 11:20:00,TYRE,[BLCHA120024R] BAN LUAR CHAOYANG CB972E 1.200 R24,<p><br></p>


# **CC : Cost Controll**

## CC1 : Summary of Working Hours

In [142]:
cc1 = kpi[['date','spk_project','company','project/name','production','move_type','category_equipment',
     'equipment_name','name','jam_produktivitas','total_jarak_tempuh','retase','standar_jarak','produksi']]

cc1.columns=['date','ts','company','project','production','move_type','category_equipment','equipment_name','name','hm','km','retase','move_type_km','produksi']

# # manipulate data agar sesuai kebutuhan
# _ = []
# for i in tqdm(cc1.date.unique().tolist()):
#     n = cc1[cc1.date == i]
#     for j in n.name.unique().tolist():
#         m = n[n.name==j]
#         count_ret = len(m)
#         hm = m.head(1).hm_engine_end - m.head(1).hm_engine_start
#         msg = m.head(1)
#         msg['hm'] = hm
#         msg['retase'] = count_ret
#         _.append(msg)
# cc1 = pd.concat(_)

# # isi hari dimana alat tidak dipekerjakan
# _ = []
# for i in tqdm(cc1.name.unique().tolist()):
#     n = cc1[cc1.name==i]
#     cat_eq = n.category_equipment.unique().tolist()[0]
#     eq_name = n.equipment_name.unique().tolist()[0]
#     name = i
#     n = date.merge(n, on='date', how='outer')
#     n['ts'].fillna('-', inplace=True)
#     n['company'].fillna('-', inplace=True)
#     n['project'].fillna('-', inplace=True)
#     n['production'].fillna('-', inplace=True)
#     n['move_type'].fillna('-', inplace=True)
#     n['category_equipment'].fillna(cat_eq, inplace=True)
#     n['equipment_name'].fillna(eq_name, inplace=True)
#     n['name'].fillna(i, inplace=True)
#     n['hm_engine_start'].fillna(0, inplace=True)
#     n['hm_engine_end'].fillna(0, inplace=True)
#     n['hm'].fillna(0, inplace=True)
#     n['retase'].fillna(0, inplace=True)
#     _.append(n)
    
# cc1 = pd.concat(_)

In [143]:
cc1['plan_hm'] = 0
cc1 = cc1.groupby(['date','company','project','production','move_type','category_equipment','equipment_name','name'])[['hm','plan_hm']].sum().reset_index()

# # pisahkan actual
# cc1_actual = cc1.drop(columns='plan_hm')
# cc1_actual['status'] = 'Actual'
# cc1_actual = cc1_actual.rename(columns={'hm':'values'})

# # pisahkan plan
# cc1_plan = cc1.drop(columns='hm')
# cc1_plan['status'] = 'Plan'
# cc1_plan = cc1_plan.rename(columns={'plan_hm':'values'})

# # gabung dengan menambahkkan kolom status
# cc1 = pd.concat([cc1_actual, cc1_plan])

# save data
# cc1.to_excel('./Report/cc dashboard 1.xlsx', index=False)

## CC2 : Owning & Operation Cost (OOC)

In [144]:
# dapatkan tabel equipment 
tbl_eq = tbl_equipment2[['company','equipment_category','equipment_model','name']]
tbl_eq['equipment_model'] = tbl_eq['equipment_model'].apply(lambda x: retext_equipment_name(x))
tbl_eq.columns = ['company','category_equipment','equipment_name','name']

# dapatkan nilai HM total\ per unit 
eq_hm = kpi.copy()
eq_hm = eq_hm.rename(columns={'jam_produktivitas':'hm','total_jarak_tempuh':'km'})
eq_hm = eq_hm.groupby(['date','project/name','production','move_type','stage','category_equipment','equipment_name','name'])[['hm','km','retase','produksi']].sum().reset_index()
eq_hm = eq_hm.merge(tbl_jarak, on='move_type', how='left').fillna(0)

In [145]:
# dapatkan dataset maintenance
maintenance = t1.copy()
maintenance = maintenance[['company','spk','broken','stage','category_equipment','equipment_name','name',
                           'requirement_type','category_maintenance','description_2','cost']]
maintenance['date'] = pd.to_datetime(maintenance['broken']).dt.date
maintenance['description2'] = maintenance['description_2'].apply(lambda x: str(x).split('] ')[-1])
maintenance['description1'] = maintenance.description2.apply(lambda x: ' '.join(str(x).split()[:2]))
maintenance.drop(columns=['description_2','broken'], inplace=True)
maintenance = maintenance[maintenance.stage=='DONE']

# dapatkan nilai tyre dengan mengamnil biaya pemakaian ban
tyre = maintenance[maintenance.description1=='BAN LUAR']
tyre = tyre[['date','company','stage','category_equipment','equipment_name','name','requirement_type','category_maintenance','description2','cost']]
tyre = tyre.groupby(['date','category_equipment','equipment_name','name'])[['cost']].sum().reset_index().rename(columns={'cost':'tyre'})

In [146]:
tyre[pd.to_datetime(tyre.date).dt.strftime('%m/%y')=='07/24']

,date,category_equipment,equipment_name,name,tyre
2884,2024-07-01,DUMP TRUCK,HINO ZY,DT-360,11710000.0
2885,2024-07-01,DUMP TRUCK,HINO ZY,DT-367,11710000.0
2886,2024-07-01,DUMP TRUCK,HINO ZY,DT-392,11710000.0
2887,2024-07-02,DUMP TRUCK,HINO ZY,DT-338,11710000.0
2888,2024-07-02,DUMP TRUCK,HINO ZY,DT-347,11710000.0
...,...,...,...,...,...
3074,2024-07-31,DUMP TRUCK,HINO ZY,DT-377,11710000.0
3075,2024-07-31,DUMP TRUCK,HINO ZY,DT-385,11710000.0
3076,2024-07-31,DUMP TRUCK,HINO ZY,DT-392,11710000.0
3077,2024-07-31,DUMP TRUCK,HINO ZY,DT-394,11710000.0


In [147]:
# dapatkan nilai rm dengan memisahkan semua biaya ganti ban
rm = maintenance[maintenance.description1!='BAN LUAR']
rm = rm[['date','stage','category_equipment','equipment_name','name','requirement_type','category_maintenance','description2','cost']]
rm.cost.fillna(0, inplace=True)
rm['date'] = rm['date'].apply(lambda x: str(x).split()[0])

# persiapkan data biaya maintenance FMC
po['description1'] = po.description.apply(lambda x: ' '.join(str(x).split('] ')[-1].split()[:2]))
fmc = po[po.description1 == 'MATERIAL FMC']
fmc['name'] = fmc.description.apply(lambda x: '-'.join(str(x).split()[1:3]))
fmc = fmc.merge(rm[['category_equipment','equipment_name','name']].drop_duplicates(), on='name', how='left')

# buat kolom disesuaikan dengan rm
fmc['requirement_type'] = 'FMC'
fmc['stage'] = 'DONE'
fmc['category_maintenance'] = 'FMC'

# recolumns fmc
fmc = fmc[['date','stage','category_equipment','equipment_name','name','requirement_type','category_maintenance','description','total']]
fmc = fmc.rename(columns={'total':'cost', 'description':'description2'})
fmc['date'] = fmc['date'].apply(lambda x: str(x).split()[0])

# buat kolom my untuk kebutuhan merge
fmc['my'] = pd.to_datetime(fmc.date).dt.strftime('%m/%y')
rm['my'] = pd.to_datetime(rm.date).dt.strftime('%m/%y')

fmc = fmc.merge(date.rename(columns={'month':'my'}), on='my', how='inner').drop(columns='date_x').rename(columns={'date_y':'date'})

fmc = fmc.ffill()
fmc['cost'] = fmc.cost / fmc.count_day
fmc = fmc.drop(columns='count_day')

# buat id
fmc['id'] = fmc.date + '#' + fmc.name
rm['id'] = rm.date + '#' + rm.name

# replace data parts dengan data fmc
rm = rm[rm['id'].isin(fmc['id'].tolist()) == False]

rm = pd.concat([rm, fmc]).drop(columns=['id','my'])
rm = rm.groupby(['date','category_equipment','equipment_name','name'])[['cost']].sum().reset_index().rename(columns={'cost':'rm'})

In [148]:
# read data detail informasi equipment
url_depresiasi = "https://docs.google.com/spreadsheets/d/1cjcBMY29etIAeQAwRDzlbpd4wExYkpvxk7s4tgVUD-w/export?format.xlsx"

try:
    os.remove('detail_depresiasi.xlsx')
except:
    None
    
output_filename = 'detail_depresiasi.xlsx'

# get the data from spreadsheet
response = requests.get(url_depresiasi)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)

# read data
dep = pd.read_excel(output_filename)
dep.columns = ['_'.join(i.lower().split()) for i in dep.columns]
dep.fillna(0, inplace=True)
dep.columns = ['company','category_equipment','equipment_name','name','date_pay','eq_age','dpp','lama_penyusutan',
               'price_aftar_salvage','depresiasi','insurance','interest','tax',]

# test saja dulu
# dep['depresiasi'] = dep['depresiasi'] * 12 * dep['lama_penyusutan']
# dep['insurance'] = dep['insurance'] * 12 * dep.lama_penyusutan
# dep['interest'] = dep['interest'] * 12 * dep.lama_penyusutan
# dep['tax'] = dep['tax'] * 12 * dep.lama_penyusutan

dep['interest'] = dep['dpp'] * dep['interest']
dep['tax'] = dep['dpp'] * dep['tax']

dep = dep.drop_duplicates()
dep = dep.groupby(['category_equipment','equipment_name','name'])[['depresiasi','interest','tax','insurance']].sum().reset_index()

In [149]:
# dapatkan nilai OVH
# employee cost
url_ovh = "https://docs.google.com/spreadsheets/d/1Xo6dCsgrSNX6pa9Q1hSnFtKzSYhdhMHNRQ4ppKnwmUk/export?format.xlsx"
try:
    os.remove('cost_ovh.xlsx')
except:
    None
    
output_filename = 'cost_ovh.xlsx'
# get the data from spreadsheet
response = requests.get(url_ovh)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)


ovh = pd.read_excel('cost_ovh.xlsx', sheet_name='Sheet8')
ovh.columns = ['name','gaji pokok','m1','m2','m3','m4','m5','m6','m7']
ovh.replace('-', 0, inplace=True)

ovh['name'] = ovh.name.apply(lambda x: str(x).split('*')[0])
ovh['name'] = ovh.name.apply(lambda x: ''.join(x.split()))

def rename_eq_detail(x):
    con = x.split('-')[0]
    if con == 'HINO12R':
        msg = 'DT-' + x.split('-')[-1]
    else:
        msg = x
    return msg

def rename_eq_detail2(x):
    con = x.split('-')[-1]
    con = int(con)
    if con > 500:
        msg = 'DT-' + x.split('-')[-1]
    else:
        msg = x
    return msg

ovh = ovh[ovh.name!='nan']
ovh['name'] = ovh.name.apply(rename_eq_detail)
ovh['name'] = ovh.name.apply(rename_eq_detail2)

ovh = ovh.replace('-', 0)

# perbaiki nama equipment
ovh['name'] = ovh['name'].apply(lambda x: ''.join(str(x).split()))
ovh['name'] = ovh['name'].apply(lambda x: str(x).split('*')[0])

# rekontruksi data struktur
_ = []
for i, j in zip(['m1','m2','m3','m4','m5','m6'], ['01/24','02/24','03/24','04/24','05/24','06/24']):
    ovh[i] = ovh[i] + ovh['gaji pokok']
    n = ovh[['name',i]].rename(columns={i:'emp_cost'})
    n['month'] = j
    _.append(n)

# gabungkan data kembali
ovh = pd.concat(_)[['name','month','emp_cost']]
ovh = ovh.drop_duplicates()
ovh = ovh.groupby(['name','month'])[['emp_cost']].sum().reset_index()

# buat data biaya konsumsi oprator dan driver
ovh_makanan = kpi[kpi['project/name'].isin(['Mining KBM','Mining BDM'])]
ovh_makanan = ovh_makanan[['date','name','jam_produktivitas']]
ovh_makanan = ovh_makanan[ovh_makanan.jam_produktivitas!=0]
ovh_makanan = ovh_makanan.drop_duplicates(subset=['date','name'])
ovh_makanan['cost_makanan'] = 13000 * 3
ovh_makanan['month'] = pd.to_datetime(ovh_makanan.date).dt.strftime('%m/%y')
ovh_makanan = ovh_makanan.groupby(['name','month'])[['cost_makanan']].sum().reset_index()

# buat id untuk merge
ovh_makanan['id'] = ovh_makanan.month + '#' + ovh_makanan.name
ovh['id'] = ovh.month + '#' + ovh.name

# merge data
ovh = ovh.merge(ovh_makanan[['id','cost_makanan']], how='left', on='id').fillna(0)
ovh['month'] = ovh.id.apply(lambda x: x.split('#')[0])
ovh['name'] = ovh.id.apply(lambda x: x.split('#')[-1])
ovh = ovh.drop(columns='id')
ovh = ovh[['name','month','cost_makanan','emp_cost']].fillna(0)
ovh['ovh'] = ovh.cost_makanan + ovh.emp_cost
ovh.drop_duplicates(inplace=True)

ovh = ovh.groupby(['name','month'])[['cost_makanan','emp_cost','ovh']].sum().reset_index()

In [150]:
# dapatkan nilai subcont

subcont = pd.DataFrame({})

In [151]:
# buat fungsi untuk membuat id
def get_id(date, name):
    date = str(date)
    name = str(name)
    msg = date + '#' + name
    return msg

eq_hm['id'] = eq_hm.apply(lambda x: get_id(x['date'], x['name']), axis=1)
rm['id'] = rm.apply(lambda x: get_id(x['date'], x['name']), axis=1)
fuel['id'] = fuel.apply(lambda x: get_id(x['date'], x['name']), axis=1)
tyre['id'] = tyre.apply(lambda x: get_id(x['date'], x['name']), axis=1)
ovh['id'] = ovh.apply(lambda x: get_id(x['month'], x['name']), axis=1)
# subcont['id'] = subcont.apply(lambda x: get_id(x['date'], x['name']), axis=1)

In [152]:
ooc = eq_hm.merge(rm[['id','rm']], on='id', how='outer')
ooc = ooc.merge(fuel[['id','fuel_cost']], on='id', how='outer').rename(columns={'fuel_cost':'fuel'})
ooc = ooc.merge(tyre[['id','tyre']], on='id', how='outer')

ooc = ooc.drop(columns=['date', 'category_equipment','equipment_name','name'])
ooc['date'] = ooc['id'].apply(lambda x: str(x).split('#')[0])
ooc['name'] = ooc['id'].apply(lambda x: str(x).split('#')[-1])

ooc = ooc.merge(tbl_eq, on='name', how='left')
ooc['category_equipment'].fillna('Tidak Diketahui', inplace=True)
ooc['equipment_name'].fillna('Tidak Diketahui', inplace=True)
ooc['name'].fillna('Tidak Diketahui', inplace=True)
ooc['production'].fillna('-', inplace=True)
ooc['project/name'].fillna('-', inplace=True)
ooc['move_type'].fillna('-', inplace=True)
ooc['stage'].fillna('-', inplace=True)
ooc.fillna(0, inplace=True)

ooc = ooc.merge(dep[['name','depresiasi','insurance','interest','tax']], on='name', how='left')
ooc = ooc[['date','project/name','production','move_type','stage','category_equipment','equipment_name','name','hm','km','produksi','rm',
           'fuel','tyre','depresiasi','insurance','tax','interest']].fillna(0)

ooc['month'] = pd.to_datetime(ooc.date).dt.strftime('%m/%y')

# buat persentase dt perbulan 
_ = []
for i in tqdm(ooc.name.unique().tolist()):
    n = ooc[ooc.name==i]
    for j in n.month.unique().tolist():
        m = n[n.month==j]
        m['p_hm'] = m.hm / m.hm.sum()
        _.append(m)
ooc = pd.concat(_)
ooc.p_hm.fillna(0, inplace=True)

# tambahkan tanggal kosong pada DT yang standby
_ = []
for i in tqdm(ooc.name.unique().tolist()):
    n = ooc[ooc.name==i]

    cat_eq = n.category_equipment.unique().tolist()[0]
    eq_name = n.equipment_name.unique().tolist()[0]
    name = n.name.unique().tolist()[0]
    
    n = n.merge(date.drop(columns='month'), on='date', how='outer')
    n['category_equipment'].fillna(cat_eq, inplace=True)
    n['equipment_name'].fillna(eq_name, inplace=True)
    n['name'].fillna(name, inplace=True)
    n['project/name'].fillna('-', inplace=True)
    n['production'].fillna('-', inplace=True)
    n['move_type'].fillna('-', inplace=True)
    n['stage'].fillna('-', inplace=True)
    n.fillna(0, inplace=True)
    _.append(n)
ooc = pd.concat(_)

# owning cost methods - 1
# scaling owning cost into size of a day per month
ooc['month'] = pd.to_datetime(ooc.date).dt.strftime('%m/%y')
ooc['depresiasi'] = ooc.depresiasi / ooc.count_day
ooc['insurance'] = ooc.insurance / ooc.count_day
ooc['tax'] = ooc.tax / ooc.count_day
ooc['interest'] = ooc.interest / ooc.count_day
ooc = ooc.drop(columns=['count_day'])

# owning cost methods - 2
# ooc['p_hm2'] = ooc.hm / ooc.hm.sum()
# ooc['depresiasi'] = ooc.depresiasi * ooc.p_hm2
# ooc['insurance'] = ooc.insurance * ooc.p_hm2
# ooc['tax'] = ooc.tax * ooc.p_hm2
# ooc['interest'] = ooc.interest * ooc.p_hm2
# ooc = ooc.drop(columns=['p_hm2'])

# buat fungsi pembuat id untuk pertemuan dengan OVH
def get_id_ovh(month, name):
    return str(month) + "#" + str(name)
    
# buat id ooc untuk pertemuan dengan OVH
ooc['id'] = ooc.apply(lambda x: get_id_ovh(x['month'], x['name']), axis=1) 
ovh['id'] = ovh.apply(lambda x: get_id_ovh(x['month'], x['name']), axis=1)

# fillna ovh
ooc = ooc.merge(ovh[['id','ovh']], on='id', how='left').fillna(0)
ooc['ovh'] = ooc.ovh * ooc.p_hm
ooc = ooc.drop(columns=['p_hm','month'])

# load datasets
ooc['equipment_name'] = ooc.equipment_name.apply(retext_equipment_name)
ooc = ooc.drop_duplicates()

# merge stander jarak
ooc = ooc.merge(tbl_jarak, on='move_type', how='left')
ooc['jarak'].fillna(0, inplace=True)

def fuel_condition(kontrak, fuel):
    if kontrak == 'Mining BDM':
        msg = 0
    else:
        msg = fuel
    return msg

ooc['fuel'] = ooc.apply(lambda x: fuel_condition(x['project/name'], x.fuel), axis=1)

ooc.to_excel('./Report/Owning & Operating Cost.xlsx', index=False)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 432/432 [00:04<00:00, 88.08it/s]


In [153]:
kpi['date'] = kpi['date'].apply(lambda x: str(x).split()[0])
cost = ooc.copy()

# buat id pertemuan
cost['id'] = cost['date'] + '#' + cost['name']
kpi['id'] = kpi['date'] + '#' + kpi['name']

# merge data
kpi = kpi.merge(cost[['id','rm','fuel','tyre','depresiasi','interest','insurance','tax','ovh']], on='id', how='left')
kpi['cost'] = kpi.rm + kpi.fuel + kpi.tyre + kpi.depresiasi + kpi.interest + kpi.insurance + kpi.tax + kpi.ovh
# kpi = kpi.drop(columns = ['id','rm','fuel','tyre','depresiasi','interest','insurance','tax'])
kpi = kpi.drop(columns = ['id'])

kpi['cost / produksi . km'] = kpi.cost / (kpi.produksi * kpi.total_jarak_tempuh)
kpi['cost / produksi . km'].replace(np.inf, 0, inplace=True)
kpi['cost / produksi . km'].fillna(0, inplace=True)

In [154]:
kpi.to_excel('./Report/task2.xlsx', index=False)

## CC3 : OOC Actual & OOC Full

In [155]:
# ooc_diff = ooc.copy()

# # buat kolom ooc
# ooc_diff['ooc'] = (ooc_diff.rm + ooc_diff.fuel + ooc_diff.tyre + ooc_diff.depresiasi + ooc_diff.insurance + ooc_diff.ovh)/ooc_diff.hm
# ooc_diff['ooc'].fillna(0, inplace=True)
# ooc_diff['ooc'].replace(np.inf, 0, inplace=True)

# # buat kolom ooc full
# ooc_diff['ooc_full'] = (ooc_diff.rm + ooc_diff.fuel + ooc_diff.tyre + ooc_diff.depresiasi + ooc_diff.insurance + ooc_diff.ovh)/ooc_diff.hm_full
# ooc_diff['ooc_full'].fillna(0, inplace=True)
# ooc_diff['ooc_full'].replace(np.inf, 0, inplace=True)

# # grouping
# _ = []
# ooc_diff = ooc_diff.groupby(['date','category_equipment','equipment_name','name'])[['ooc','ooc_full']].sum().reset_index()
# for i in ['ooc','ooc_full']:
#     n = ooc_diff[['date','category_equipment','equipment_name','name',i]]
#     n['status'] = i
#     n = n.rename(columns={i:'values'})
#     _.append(n)

# ooc_diff = pd.concat(_)
# ooc_diff.to_excel('./Report/cc dashboard 3.xlsx', index=False)

## CC4 : Production * OOC

In [156]:
# cc41 = cc1[cc1.status=='Actual'].drop(columns='status').rename(columns={'values':'hm'})

In [157]:
# cc42 = ooc.groupby(['name'])[['hm','rm','fuel','tyre','depresiasi','insurance','interest','ovh']].sum().reset_index()
# cc42['ooc'] = (cc42.rm + cc42.fuel + cc42.tyre + cc42.depresiasi + cc42.insurance + cc42.interest + cc42.ovh)/cc42.hm
# cc42.replace(np.inf, 0, inplace=True)
# cc42.fillna(0, inplace=True)

In [158]:
# cc4 = cc41.merge(cc42[['name','ooc']], on='name', how='left')
# cc4['actual_cost'] = cc4.hm * cc4.ooc

In [159]:
# # save data
# cc4.to_excel('./Report/cc dashboard 4.xlsx', index=False)

## CC5 : Evaluating & Budgeting

In [160]:
# siapkan data ooc
cc5 = ooc.groupby(['equipment_name'])[['hm','rm','fuel','tyre','depresiasi','insurance','interest','ovh','tax']].sum().reset_index()

cc5.replace(np.inf, 0, inplace=True)
cc5.fillna(0, inplace=True)

cc5['ooc'] = (cc5.rm + cc5.fuel + cc5.tyre + cc5.depresiasi + cc5.insurance + cc5.interest + cc5.ovh)/cc5.hm
cc5['operating_cost'] = (cc5.rm  + cc5.fuel + cc5.tyre) / cc5.hm

# dapatkan masing-masing parameter cost per hm
for i in ['rm','fuel','tyre','depresiasi','insurance','interest','ovh','tax']:
    name = i + '/hm'
    cc5[name] = cc5[i] / cc5['hm']

cc5.fillna(0, inplace=True)
cc5.replace(np.inf, 0, inplace=True)
cc5.drop(columns='ovh', inplace=True)

In [161]:
# data plan MPE
url_standar_ovh = "https://docs.google.com/spreadsheets/d/1Oj7X0CU6l754u__yfBcOVxlVAlFRvSrsbj4c9hc9f6k/export?format.xlsx"

try:
    os.remove('standar_ovh.xlsx')
except:
    None
output_filename = 'standar_ovh.xlsx'

# get the data from spreadsheet
response = requests.get(url_standar_ovh)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)

# read rate perusahaan
rate_perusahaan = pd.read_excel(output_filename, sheet_name='Perusahaan MPE')
rate_perusahaan.columns = ['_'.join(i.lower().split()) for i in rate_perusahaan.columns]

# read rate gaji driver dan oprator
rate_emp = pd.read_excel(output_filename, sheet_name='Karyawan')
rate_emp.columns = ['_'.join(i.lower().split()) for i in rate_emp.columns]
rate_emp = rate_emp.groupby(['multiplication_type','equipment_name'])[['rate']].mean().reset_index()

# read gaji pokok
ovh_pokok = pd.read_excel(output_filename, sheet_name='OVH Pokok')
ovh_pokok.columns=['name','gaji_pokok']

ovh_pokok['name2'] = ovh_pokok.name.apply(lambda x: str(x).split('*')[0])
ovh_pokok['name2'] = ovh_pokok.name2.apply(lambda x: ''.join(x.split()))

ovh_pokok = ovh_pokok[ovh_pokok.name2!='nan']
ovh_pokok['name2'] = ovh_pokok.name2.apply(rename_eq_detail)
ovh_pokok['name2'] = ovh_pokok.name2.apply(rename_eq_detail2)

# grouping
ovh_pokok = ovh_pokok.groupby(['name2'])[['gaji_pokok']].sum().reset_index()
ovh_pokok = ovh_pokok.merge(tbl_eq[['equipment_name','name']].drop_duplicates(), left_on='name2', right_on='name', how='left').drop(columns='name2')[['equipment_name','name','gaji_pokok']]

In [162]:
# data plan MPE
url_plan_mpe = 'https://docs.google.com/spreadsheets/d/1RZvzn28cvdzdQSPAdNHfVqy3tPoUUhkgjeVFAzGSf8c/export?format.xlsx'

try:
    os.remove('plan_mpe.xlsx')
except:
    None
output_filename = 'plan_mpe.xlsx'

# get the data from spreadsheet
response = requests.get(url_plan_mpe)
if response.status_code == 200:
    with open(output_filename, "wb") as f:
        f.write(response.content)

# read data plan hm
plan_hm = pd.read_excel(output_filename, sheet_name="plan")

In [163]:
plan_hm.equipment_name.replace('HONGYAN KINKAN430 1', 'HONGYAN KINKAN430', inplace=True)
plan_hm.equipment_name.replace('HONGYAN KINKAN430 2', 'HONGYAN KINKAN430', inplace=True)
plan_hm.equipment_name.replace('KOMATSU PC300 1', 'KOMATSU PC300', inplace=True)
plan_hm.equipment_name.replace('KOMATSU PC300 2', 'KOMATSU PC300', inplace=True)

plan_hm_columns = plan_hm.columns.tolist()
plan_hm = plan_hm[1:]
plan_hm.columns = plan_hm_columns

plan_hm = plan_hm[['period','date','company','kontrak','production','category_equipment','equipment_name','% pa','alokasi',
         'hm_per_unit','fuel_per_unit','productivity per hour (ton/hour)','plan_hm','plan_fuel','plan_retase','plan_produksi','budget_cost']]

plan_hm = plan_hm.rename(columns={'fuel_per_unit':'fuel_per_hour/retase','productivity per hour (ton/hour)':'productivity (ton/hour)','% pa':'pa'})

plan_hm.plan_retase.fillna(0, inplace=True)
plan_hm = plan_hm.drop_duplicates(subset=['period','date','kontrak','production','category_equipment','equipment_name'])
plan_hm['hm_per_unit'] = plan_hm['hm_per_unit'].astype(float)
plan_hm['date'] = plan_hm['date'].apply(lambda x: str(x).split()[0])

In [164]:
# plan_hm_quarry = plan_hm[plan_hm.production=='Overburden']
# plan_hm_quarry['production'] = 'Quarry'

# for i in plan_hm_quarry.select_dtypes('number'):
#     plan_hm_quarry[i] = plan_hm_quarry[i] * 0.1

# plan_hm = pd.concat([plan_hm, plan_hm_quarry])

# def reduce_plan_ob(prod, x):
#     if prod == 'Overburden':
#         msg = x - (x * 0.1)
#     else:
#         msg = x
#     return msg

# for i in ['hm_per_unit','fuel_per_hour/retase','productivity (ton/hour)','plan_hm','plan_fuel','plan_retase','plan_produksi','budget_cost']:
#     plan_hm[i] = plan_hm.apply(lambda x: reduce_plan_ob(x.production, x[i]), axis=1)

In [165]:
plan_hm = plan_hm.merge(cc5[['equipment_name','ooc','operating_cost']], on='equipment_name', how='left').rename(columns={'operating_cost':'operating_per_hm'})

plan_hm['budget_ooc'] = plan_hm.plan_hm * plan_hm.ooc
plan_hm['budget_operating'] = plan_hm.plan_hm * plan_hm.operating_per_hm
plan_hm['plan_fuel'] = plan_hm.plan_fuel * fuel_price.price2.tolist()[-1]
plan_hm['budget_cost'] = plan_hm.budget_cost + plan_hm.plan_fuel
plan_hm.fillna(0, inplace=True)

plan_hm['date'] = plan_hm['date'].apply(lambda x: str(x).split()[0])

In [166]:
# read data prodcution
plan_production = pd.read_excel(output_filename, sheet_name="production").drop(columns='company')

plan_production['plan_produksi'] = plan_production['plan_produksi'].apply(lambda x: ''.join('.'.join(str(x).split('.')).split(',')))
plan_production['plan_produksi'] = plan_production['plan_produksi'].astype(float)

plan_production['actual_produksi'] = plan_production['actual_produksi'].apply(lambda x: ''.join('.'.join(str(x).split('.')).split(',')))
plan_production['actual_produksi'] = plan_production['actual_produksi'].astype(float)

plan_production.actual_produksi.fillna(0, inplace=True)
plan_production = plan_production.drop_duplicates()

In [167]:
class_date = pd.read_excel(output_filename, sheet_name='tbl klasifikasi minggu')

# read data actual mpe
actual_mpe = pd.read_excel(output_filename, sheet_name="actual")

# transformasi dan cleaning data
actual_mpe['hm'] = actual_mpe['hm'].fillna(0)
actual_mpe['hm'] = actual_mpe['hm'].apply(lambda x: ''.join(str(x).split(',')))
actual_mpe.hm.replace('-', 0, inplace=True)
actual_mpe['hm'] = actual_mpe.hm.astype(float)

actual_mpe['fuel'] = actual_mpe['fuel'].fillna(0)
actual_mpe['fuel'] = actual_mpe['fuel'].apply(lambda x: ''.join(str(x).split(',')))
actual_mpe.fuel.replace('-', 0, inplace=True)
actual_mpe['fuel'] = actual_mpe.fuel.astype(float)

actual_mpe.tonase.fillna(0, inplace=True)
actual_mpe['tonase'] = actual_mpe['tonase'].apply(lambda x: ''.join(str(x).split(',')))
actual_mpe.tonase.replace('-', 0, inplace=True)
actual_mpe['tonase'] = actual_mpe.tonase.astype(float)

actual_mpe.retase.fillna(0, inplace=True)
actual_mpe['retase'] = actual_mpe['retase'].apply(lambda x: ''.join(str(x).split(',')))
actual_mpe.retase.replace('-', 0, inplace=True)
actual_mpe['retase'] = actual_mpe.retase.astype(float)

# transformasi data harga fuel dan total biaya fuel
actual_mpe['price_fuel'] = fuel_price.price2.tolist()[-1]
actual_mpe = actual_mpe[actual_mpe.fuel!=0]
actual_mpe['fuel'] = actual_mpe.fuel * actual_mpe.price_fuel
actual_mpe.drop(columns=['price_fuel','equipment_mpe','company','Unnamed: 3'], inplace=True)

# kondisi untuk Mining BDM fuel ditanggung Owner
actual_mpe['fuel'] = actual_mpe.apply(lambda x: 0 if x.kontrak == 'Mining BDM' else x.fuel, axis=1)

In [168]:
# read data aktual
actual = ooc.copy()

# filter menggunakan tanggal plan MPE
actual = actual[actual.date.isin(plan_hm.date.unique().tolist())]

# filter menggunakan Equipment Name yang ada pada MPE
actual = actual[actual.name.isin(actual_mpe.name.unique().tolist())]

# ambil semua Unit termasuk
actual = actual[actual['project/name'].isin(['Mining KBM','Mining BDM'])]

# buat kolom actual_cost, operarting_cost dan owning_cost
# actual['actual_cost'] = actual.rm + actual.fuel + actual.tyre + actual.depresiasi + actual.insurance + actual.tax + actual.interest + actual.ovh
# actual['operating_cost'] = actual.rm + actual.fuel + actual.tyre
# actual['owning_cost'] = actual.depresiasi + actual.insurance + actual.tax + actual.interest + actual.ovh

actual = actual.drop(columns = ['depresiasi','insurance','tax','interest','ovh'])

# grouping data
actual = actual.groupby(['date','project/name','production','equipment_name','name'])[['hm','km','jarak','rm','tyre']].sum().reset_index()

# rename columns
actual = actual.rename(columns={
    'produksi':'actual_produksi',
    'hm':'actual_hm',
    'project/name':'kontrak'
})

# buat fungsi untuk klasifikasi production antara Aktual terhadap data MPE
def transform_production(x):
    if x in ['Stripping Overburden']:
        msg = 'Overburden'
    elif x in ['Hauling Ore','Ore Getting']:
        msg = 'Ore Mined'
    elif x in ['Quarry Mined']:
        msg = 'Quarry'
    return msg

actual = actual[actual.production.isin(['Stripping Overburden','Hauling Ore','Ore Getting','Quarry Mined'])]
actual['production'] = actual.production.apply(transform_production)

actual['date'] = actual['date'].apply(lambda x: str(x).split()[0])
class_date['date'] = class_date['date'].apply(lambda x: str(x).split()[0])

actual = actual.merge(class_date[['date','period']], on='date', how='left')
actual = actual.groupby(['period','kontrak','production','equipment_name','name'])[['actual_hm','km','jarak','rm','tyre']].sum().reset_index()

# merge
actual['id'] = actual.period + '#' + actual.kontrak + '#' + actual.production + '#' + actual.equipment_name + '#' + actual.name
actual_mpe['id'] = actual_mpe.period + '#' + actual_mpe.kontrak + '#' + actual_mpe.production + '#' + actual_mpe.equipment_name + '#' + actual_mpe.name

actual = actual_mpe[['id','hm','fuel','tonase','retase']].merge(actual[['id','actual_hm','rm','tyre','km','jarak']], on='id', how='left')
actual['period'] = actual['id'].apply(lambda x: str(x).split('#')[0])
actual['kontrak'] = actual['id'].apply(lambda x: str(x).split('#')[1])
actual['production'] = actual['id'].apply(lambda x: str(x).split('#')[2])
actual['equipment_name'] = actual['id'].apply(lambda x: str(x).split('#')[3])
actual['name'] = actual['id'].apply(lambda x: str(x).split('#')[4])

actual.drop(columns=['id'], inplace=True)

actual = actual[['period','kontrak','production','equipment_name','name','hm','km','jarak','actual_hm','tonase','retase','rm','fuel','tyre',]]
actual['tonase'].fillna(0, inplace=True)
actual['retase'].fillna(0, inplace=True)
actual['fuel'].fillna(0, inplace=True)

# buat fungsi untuk inputkan hm mpe dengan hm node3 jika hm mpe nol
def replace_hm_condition(x, y):
    if x == 0:
        msg = y
    else:
        msg = x
    return msg

actual['hm'] = actual.apply(lambda x: replace_hm_condition(x.hm, x.actual_hm), axis=1)
actual.drop(columns='actual_hm', inplace=True)

actual = actual.merge(cc5[['equipment_name','depresiasi/hm','insurance/hm','interest/hm','ovh/hm','tax/hm']], on='equipment_name', how='left')
actual['depresiasi'] = actual.hm * actual['depresiasi/hm']
actual['insurance'] = actual.hm * actual['insurance/hm']
actual['interest'] = actual.hm * actual['interest/hm']
actual['tax'] = actual.hm * actual['tax/hm']
actual.drop(columns = ['depresiasi/hm','insurance/hm','interest/hm','ovh/hm','tax/hm'], inplace=True)

# buat kolom gaji tambahan
actual = actual.merge(rate_emp[['multiplication_type','equipment_name','rate']], on='equipment_name', how='left')
actual['gaji_tambahan'] = actual.apply(lambda x: x.tonase * x.rate if x.multiplication_type == 'Tonase' else x.hm * x.rate, axis=1)
actual.fillna(0, inplace=True)

count_day_per_week = class_date.groupby(['period'])[['date']].count().reset_index().rename(columns={'date':'count_day'})
actual = actual.merge(count_day_per_week[['period','count_day']], on='period', how='left')

# buat id untuk actual
actual['id'] = actual.period + '#' + actual.kontrak

# buat id untuk actual
actual['id2'] = actual.kontrak + '#' + actual.name

_ = []
for i in actual.id.unique().tolist():
    data = actual[actual.id == i]
    data['p_hm'] = data.hm / data.hm.sum()
    _.append(data)

actual = pd.concat(_)

In [169]:
gaji_bulan = actual[['period','kontrak','equipment_name','name']]
gaji_bulan['month'] = gaji_bulan.period.apply(lambda x: str(x).split()[-1])
gaji_bulan = gaji_bulan[['month','kontrak','equipment_name','name']].drop_duplicates()
gaji_bulan = gaji_bulan.merge(ovh_pokok[['name','gaji_pokok']], on='name', how='left')
gaji_bulan = gaji_bulan.merge(date[['month','count_day']].drop_duplicates(), on='month', how='left')
gaji_bulan['gaji_pokok_harian'] = gaji_bulan.gaji_pokok / gaji_bulan.count_day

# buat id
gaji_bulan['id2'] = gaji_bulan.kontrak + '#' + gaji_bulan.name

# merge dengan data actual
gaji_bulan2 = actual[['period','kontrak','name','id2','count_day']].drop_duplicates()
gaji_bulan2 = gaji_bulan2.merge(gaji_bulan[['id2','gaji_pokok_harian']].drop_duplicates(), on='id2', how='left')
gaji_bulan2['gaji_pokok'] = gaji_bulan2.gaji_pokok_harian * gaji_bulan2.count_day
gaji_bulan2 = gaji_bulan2.groupby(['period','kontrak'])[['gaji_pokok']].sum().reset_index()

gaji_bulan2['id'] = gaji_bulan2.period + '#' + gaji_bulan2.kontrak

In [170]:
biaya_makan = actual[['period','kontrak','name']].drop_duplicates()
biaya_makan = biaya_makan.groupby(['period','kontrak'])[['name']].count().reset_index().rename(columns={'name':'count_unit'})
biaya_makan['harga_makanan'] = 12500
biaya_makan['jumlah_makan_perhari'] = 3
biaya_makan['biaya_makan'] = biaya_makan.count_unit * biaya_makan.harga_makanan * biaya_makan.jumlah_makan_perhari
biaya_makan['id'] = biaya_makan.period + '#' + biaya_makan.kontrak

In [171]:
actual = actual.merge(gaji_bulan2[['id','gaji_pokok']].drop_duplicates(), on='id', how='left')
actual['gaji_pokok'] = actual['gaji_pokok'] * actual['p_hm']

actual = actual.merge(biaya_makan[['id','biaya_makan']].drop_duplicates(), on='id', how='left')
actual['biaya_makan'] = actual['biaya_makan'] * actual['p_hm']

actual = actual.drop(columns=['id','id2','p_hm','count_day'])

In [172]:
actual['ovh'] = actual.gaji_tambahan + actual.gaji_pokok + actual.biaya_makan

In [173]:
a = plan_hm.copy()
a = a.groupby(['period','kontrak','production','equipment_name'])[['alokasi','plan_produksi','plan_hm','plan_fuel','budget_cost']].sum().reset_index()
a['id'] = a.period + '#' + a.equipment_name + '#' + a.production + '#' + a.kontrak

b = actual.copy()
b = b.groupby(['period','kontrak','production','equipment_name'])[['hm','km','jarak','tonase','retase','rm','fuel','tyre','depresiasi','insurance','tax','interest','ovh']].sum().reset_index()
b['id'] = b.period + '#' + b.equipment_name + '#' + b.production + '#' + b.kontrak

evaluasi1 = a[['id','plan_produksi','plan_hm','plan_fuel','budget_cost']].merge(b[['id','hm','km','jarak','tonase','retase','rm','fuel',
                                                                                  'tyre','depresiasi','insurance','tax','interest','ovh']], on='id', how='outer')

evaluasi1.fillna(0, inplace=True)
evaluasi1['period'] = evaluasi1.id.apply(lambda x: str(x).split('#')[0])
evaluasi1['equipment_name'] = evaluasi1.id.apply(lambda x: str(x).split('#')[1])
evaluasi1['production'] = evaluasi1.id.apply(lambda x: str(x).split('#')[2])
evaluasi1['kontrak'] = evaluasi1.id.apply(lambda x: str(x).split('#')[-1])
evaluasi1.drop(columns='id', inplace=True)

# klasifikasi jenis biaya
evaluasi1['actual_cost'] = evaluasi1.fuel + evaluasi1.rm + evaluasi1.tyre + evaluasi1.depresiasi + evaluasi1.insurance + evaluasi1.tax + evaluasi1.interest + evaluasi1.ovh
evaluasi1['operating_cost'] = evaluasi1.fuel + evaluasi1.rm + evaluasi1.tyre
evaluasi1['owning_cost'] = evaluasi1.depresiasi + evaluasi1.insurance + evaluasi1.tax + evaluasi1.interest + evaluasi1.ovh

evaluasi1 = evaluasi1[['period','kontrak','production','equipment_name','plan_hm','hm','km','jarak','retase','plan_produksi','tonase','plan_fuel','fuel','rm','tyre','depresiasi','insurance','tax',
                       'interest','ovh','actual_cost','budget_cost','operating_cost','owning_cost']]

evaluasi1.production.replace('Quarry','Overburden', inplace=True)

In [174]:
# siapkan nilai standar ratio menggunakan rata-rata
rata_ratio = ooc.copy()
rata_ratio['my'] = pd.to_datetime(rata_ratio.date).dt.strftime('%m/%y')
# rata_ratio = rata_ratio[rata_ratio.production!='-']

# rata_ratio = rata_ratio.groupby(['my','category_equipment','equipment_name'])[['hm','jarak','produksi','rm','fuel','tyre']].sum().reset_index()
rata_ratio = rata_ratio.groupby(['equipment_name'])[['hm','jarak','produksi','rm','fuel','tyre']].sum().reset_index()

# rata_ratio = rata_ratio[rata_ratio.category_equipment.isin(['DUMP TRUCK','HEAVY EQUIPMENT'])]

penyebut = ['hm','jarak','produksi']
pembilang = ['rm','fuel','tyre']

ratio_cols = []
for i in penyebut:
    for j in pembilang:
        
        if i == 'jarak':
            name_cols = 'ratio (%s/%s) avg'%(j, 'km')
        else:
            name_cols = 'ratio (%s/%s) avg'%(j, i)
        
        rata_ratio[name_cols] = rata_ratio[j] / rata_ratio[i]
        ratio_cols.append(name_cols)

rata_ratio.fillna(0, inplace=True)
rata_ratio.replace(np.inf, 0, inplace=True)
rata_ratio = rata_ratio[['equipment_name'] + ratio_cols]

In [175]:
def get_nilai_tolerant(values):
    mean = np.median(values)
    std = np.std(values)
    msg = mean + (mean * 0.25)
    return msg

rata_ratio_ = []
for i in rata_ratio.equipment_name.unique():
    n = rata_ratio[rata_ratio.equipment_name==i][['equipment_name'] + ratio_cols]
    m = n.head(1)
    
    _ = []
    for j in ratio_cols:
        m[j] = get_nilai_tolerant(n[j])

    rata_ratio_.append(m)

rata_ratio = pd.concat(rata_ratio_)

In [176]:
# merge data rata-rata ratio dengan evaluasi1
evaluasi1 = evaluasi1.merge(rata_ratio, on='equipment_name', how='left')

In [177]:
evaluasi2 = evaluasi1.groupby(['period','kontrak','production'])[['plan_hm','hm','budget_cost','actual_cost','operating_cost','owning_cost','rm','plan_fuel','fuel','tyre',
                                                                  'depresiasi','insurance','interest','tax','ovh','tonase']].sum().reset_index()
evaluasi2['id'] = evaluasi2.period + '#' + evaluasi2.production + '#' + evaluasi2.kontrak

a = plan_production.copy()
a = a.groupby(['period','kontrak','production'])[['plan_produksi','actual_produksi']].sum().reset_index()
a['id'] = a.period + '#' + a.production + '#' + a.kontrak

evaluasi2 = a[['id','plan_produksi','actual_produksi']].merge(evaluasi2[['id','plan_hm','hm','budget_cost','actual_cost','operating_cost','owning_cost','rm','plan_fuel','fuel','tyre',
                                                       'depresiasi','insurance','interest','tax','ovh']], on='id', how='outer')
evaluasi2 = evaluasi2.rename(columns={'plan_produksi':'plan_tonase'})

evaluasi2.fillna(0, inplace=True)
evaluasi2['period'] = evaluasi2['id'].apply(lambda x: str(x).split('#')[0])
evaluasi2['production'] = evaluasi2['id'].apply(lambda x: str(x).split('#')[1])
evaluasi2['kontrak'] = evaluasi2['id'].apply(lambda x: str(x).split('#')[2])
evaluasi2.drop(columns='id', inplace=True)

evaluasi2 = evaluasi2.rename(columns={'actual_produksi':'tonase'})

evaluasi2 = evaluasi2[['period','kontrak','production','plan_tonase','tonase','plan_hm','hm','budget_cost','actual_cost','operating_cost',
           'owning_cost','plan_fuel','fuel','rm','tyre','depresiasi','insurance','interest','tax','ovh']]

evaluasi2['tonase'] = evaluasi2.apply(lambda x: 0 if x.production == 'Quarry' else x.tonase, axis=1)
evaluasi2['plan_tonase'] = evaluasi2.apply(lambda x: 0 if x.production == 'Quarry' else x.plan_tonase, axis=1)

evaluasi2.production.replace('Quarry','Overburden', inplace=True)

In [178]:
evaluasi3 = evaluasi1.copy()[['period','kontrak','production','equipment_name','owning_cost','operating_cost']]
_ = []
for i in ['owning_cost','operating_cost']:
    n = evaluasi3.copy()
    n = n[['period','kontrak','production','equipment_name',i]].rename(columns={i:'values'})
    n['status'] = i
    _.append(n)

evaluasi3 = pd.concat(_)

In [179]:
evaluasi3['profit_margin'] = 0.3

In [180]:
_ = []
for i in evaluasi3.kontrak:
    for j in evaluasi3.period:
        data = evaluasi3[(evaluasi3.kontrak==i)&(evaluasi3.period==j)]
        produksi = evaluasi2[(evaluasi2.kontrak==i)&(evaluasi2.period==j)]

        plan_ob = produksi[produksi.production=='Overburden']['plan_tonase'].unique().tolist()[0]
        plan_ore = produksi[produksi.production=='Ore Mined']['plan_tonase'].unique().tolist()[0]

        actual_ob = produksi[produksi.production=='Overburden']['tonase'].unique().tolist()[0]
        actual_ore = produksi[produksi.production=='Ore Mined']['tonase'].unique().tolist()[0]

        data['plan_ob'] = plan_ob
        data['actual_ob'] = actual_ob
        data['plan_ore'] = plan_ore
        data['actual_ore'] = actual_ore
        break

In [181]:
cost = data[data.status=='operating_cost']['values'].sum()

act_ore = data['actual_ore'].unique().tolist()[0]
pln_ore = data['plan_ore'].unique().tolist()[0]

# Parameter Omset
pm = data['profit_margin'].unique().tolist()[0]
rate = 0.13
jarak = 25
kurs = 16000

# Produksi Tonase agar Profit Margin 30%
standar_ore = cost / ((1-pm)*(rate*jarak*kurs))

In [182]:
print('1. Plan Ore :', rupiah(pln_ore).split(',')[0].split()[-1])
print('2. Actual Ore :', rupiah(act_ore).split(',')[0].split()[-1])
print('3. Standar Ore with Profit Margin 30% :', rupiah(standar_ore).split(',')[0].split()[-1])

1. Plan Ore : 14.876
2. Actual Ore : 10.572
3. Standar Ore with Profit Margin 30% : 14.258


In [183]:
__ = evaluasi1[evaluasi1.equipment_name.isin(['HINO ZY','HONGYAN KINKAN430','KOMATSU HM400'])]
_ = []
for i in __.period.unique():
    for j in __.kontrak.unique():
        for k in __.production.unique():
            data = __[__.period == i]
            data = data[data.kontrak == j]
            data = data[data.production == k]
            data = data.groupby(['period','kontrak','production','equipment_name'])[['plan_hm','hm','rm','tyre','fuel','depresiasi','insurance','tax','interest',
                                                                                     'ovh','budget_cost','operating_cost','owning_cost']].sum().reset_index()
            data['p_hm_actual'] = data.hm / data.hm.sum()
            data['p_hm_plan'] = data.plan_hm / data.plan_hm.sum()
            _.append(data)

data = pd.concat(_)
data['id'] = data.period + '#' + data.kontrak + '#' + data.production

# tarik data merge
_ = plan_production
_['id'] = _.period + '#' + _.kontrak + '#' + _.production
_ = _.groupby(['id'])[['plan_produksi','actual_produksi']].sum().reset_index()

# merge data
data = data.merge(_, on='id', how='left').fillna(0).drop(columns='id')
data['plan_produksi'] = data.p_hm_plan * data.plan_produksi
data['tonase'] = data.p_hm_actual * data.actual_produksi
data = data.drop(columns=['p_hm_plan','p_hm_actual','actual_produksi'])
evaluasi4 = data.copy()

In [184]:
evaluasi1.to_excel('./Evaluasi Plan/W1 W2 Juli 2024 1.xlsx', index=False)
evaluasi2.to_excel('./Evaluasi Plan/W1 W2 Juli 2024 2.xlsx', index=False)
evaluasi3.to_excel('./Evaluasi Plan/W1 W2 Juli 2024 3.xlsx', index=False)
evaluasi4.to_excel('./Evaluasi Plan/W1 W2 Juli 2024 4.xlsx', index=False)

# **EDA : Exploratory Data Analyst - 2**

## Task 1 : Menentukan keputusan yang tepat saat penambahan 5 DT ke Kontrak KBM

In [186]:
# siapkan data OOC untuk kedua Equipment yang akan dialokasikan
ooc_t1 = cc5[cc5.equipment_name.isin(['HONGYAN KINKAN430','HINO ZY','HINO ZS'])]

In [187]:
# siapkan data actual pada Mining KBM untuk period bulan Juli 2024
t1 = evaluasi1[evaluasi1.kontrak=='Mining KBM']
t1 = t1.groupby(['kontrak','production','equipment_name'])[t1.select_dtypes('number').columns.tolist()].sum().reset_index()

In [188]:
# data plan mpe
plan_t1 = plan_hm[plan_hm.kontrak=='Mining KBM'].groupby(['equipment_name'])[['plan_hm','plan_fuel','plan_retase','plan_produksi','budget_cost']].sum().reset_index()
plan_t1 = plan_t1.merge(plan_hm[plan_hm.kontrak=='Mining KBM'][['equipment_name','alokasi']].drop_duplicates(), on='equipment_name', how='left')

# # akumulasikan plan menjadi per unit dalam 1 bulan
plan_t1['plan_hm'] = plan_t1.plan_hm / plan_t1.alokasi
plan_t1['plan_fuel'] = plan_t1.plan_fuel / plan_t1.alokasi
plan_t1['plan_retase'] = plan_t1.plan_retase / plan_t1.alokasi
plan_t1['plan_produksi'] = plan_t1.plan_produksi / plan_t1.alokasi
plan_t1['budget_cost'] = plan_t1.budget_cost / plan_t1.alokasi

# ambil hanya equpment target
plan_t1 = plan_t1[plan_t1.equipment_name.isin(['HONGYAN KINKAN430','HINO ZY'])]
plan_t1 = plan_t1.drop(columns=['alokasi'])

# ambil nilai ooc dari actual data terjadi di bulan juli 2024
act_t1 = evaluasi1[(evaluasi1.equipment_name.isin(['HONGYAN KINKAN430','HINO ZY']))].groupby(['equipment_name'])[['hm','operating_cost','owning_cost']].sum().reset_index()
act_t1['ooc'] = (act_t1.operating_cost + act_t1.owning_cost)/act_t1.hm

# merger data
plan_t1 = plan_t1.merge(act_t1[['equipment_name','ooc']], on='equipment_name', how='left')
plan_t1['predict_cost'] = plan_t1.plan_hm * plan_t1.ooc

In [189]:
# dapatkan nilai plan produksi untuk mining kbm dalam period bulan juli 2024
plan_produksi_t1 = evaluasi2[(evaluasi2.kontrak=='Mining KBM') & (evaluasi2.production=='Ore Mined')].groupby(['production'])[['plan_tonase','tonase']].sum().reset_index()

In [190]:
cost = t1.actual_cost.sum()
produksi = plan_produksi_t1.tonase.sum()

kurs = 14848
rate = 10.5

income = produksi * kurs * rate
profit = income - cost

pm  = np.round(profit / income, 3)

In [191]:
print('1. Actual Cost :', rupiah(cost).split(',')[0])
print('2. Actual Produksi :', rupiah(produksi).split(',')[0].split()[-1])
print('3. Income :', rupiah(income).split(',')[0])
print('4. Profit :', rupiah(profit).split(',')[0])
print('5. Profit Margin :', pm)

1. Actual Cost : Rp 11.962.501.075
2. Actual Produksi : 94.399
3. Income : Rp 14.717.298.624
4. Profit : Rp 2.754.797.548
5. Profit Margin : 0.187


In [192]:
plan_t1['plan_hm'] = 157.4676
plan_t1['ton/hm'] = [21,28]
plan_t1['plan_produksi'] = plan_t1['ton/hm'] * plan_t1.plan_hm

In [193]:
# siapkan data simulasi
data_simulasi = pd.DataFrame({
    'HONGYAN KINKAN430':[5,4,3,2,1,0,0],
    'HINO ZY':[0,1,2,3,4,5,0]
})

# buat perulangan
_ = []
for i in range(len(data_simulasi)):
    data = data_simulasi[data_simulasi.index==i]

    hy = data['HONGYAN KINKAN430'][i]
    zy = data['HINO ZY'][i]
    name = '%sHY + %sZY'%(hy, zy)

    alokasi_n = [zy, hy]

    x = plan_t1.copy()
    x['alokasi'] = alokasi_n
    x['plan_produksi'] = x['plan_produksi'] * x['alokasi']
    
    # x['budget_cost'] = x['budget_cost'] * x.alokasi
    x['predict_cost'] = x['predict_cost'] * x['alokasi']

    # definisikan untuk parameter sebelum ada ketambahan alat
    cost = 11820155462.507866
    produksi = 94399.75
    kurs = 14848
    rate = 10.5
    income = 14717298624.0
    profit = 2897143161.492134
    pm = 0.197

    # olah data ketika terjadi penambahan alat
    cost_a = x.predict_cost.sum() + cost
    # cost_a = x.budget_cost.sum() + cost
    
    produksi_a = x.plan_produksi.sum() + produksi
    income_a = (produksi_a * kurs * rate)
    profit_a = income_a - cost_a
    pm_a = profit_a / income_a

    msg = pd.DataFrame({
        'cost':[cost_a],
        'produksi':[produksi_a], 
        'income':[income_a],
        'profit':[profit_a],
        'pm':[pm_a]
    })

    msg['condition'] = name
    _.append(msg)

report = pd.concat(_)[['condition','cost','produksi','income','profit','pm']]
report.sort_values('pm', ascending=False).reset_index(drop=True).to_excel('Kondisi jika menambah DT pada lokasi KBM.xlsx', index=False)
report = report.sort_values('pm', ascending=False).reset_index(drop=True)
report

,condition,cost,produksi,income,profit,pm
0,5HY + 0ZY,1.254684e+10,116445.2140,1.815427e+10,5.607438e+09,0.308877
1,4HY + 1ZY,1.249542e+10,115342.9408,1.798243e+10,5.487008e+09,0.305132
2,3HY + 2ZY,1.244400e+10,114240.6676,1.781058e+10,5.366579e+09,0.301314
3,2HY + 3ZY,1.239258e+10,113138.3944,1.763873e+10,5.246149e+09,0.297422
4,1HY + 4ZY,1.234116e+10,112036.1212,1.746688e+10,5.125719e+09,0.293454
5,0HY + 5ZY,1.228974e+10,110933.8480,1.729503e+10,5.005290e+09,0.289406
6,0HY + 0ZY,1.182016e+10,94399.7500,1.471730e+10,2.897143e+09,0.196853


In [194]:
a = report[report.index==6]

In [195]:
# siapkan data simulasi
data_simulasi = pd.DataFrame({
    'HONGYAN KINKAN430':[5,4,3,2,1,0,0],
    'HINO ZY':[0,1,2,3,4,5,0]
})

# buat perulangan
_ = []
for i in range(len(data_simulasi)):
    data = data_simulasi[data_simulasi.index==i]

    hy = data['HONGYAN KINKAN430'][i]
    zy = data['HINO ZY'][i]
    name = '%sHY + %sZY'%(hy, zy)

    alokasi_n = [zy, hy]

    x = plan_t1.copy()
    x['alokasi'] = alokasi_n
    x['plan_produksi'] = x['plan_produksi'] * x['alokasi']
    
    x['budget_cost'] = x['budget_cost'] * x.alokasi
    # x['predict_cost'] = x['predict_cost'] * x['alokasi']

    # definisikan untuk parameter sebelum ada ketambahan alat
    cost = 11820155462.507866
    produksi = 94399.75
    kurs = 14848
    rate = 10.5
    income = 14717298624.0
    profit = 2897143161.492134
    pm = 0.197

    # olah data ketika terjadi penambahan alat
    # cost_a = x.predict_cost.sum() + cost
    cost_a = x.budget_cost.sum() + cost
    
    produksi_a = 110151
    income_a = (produksi_a * kurs * rate)
    profit_a = income_a - cost_a
    pm_a = profit_a / income_a

    msg = pd.DataFrame({
        'cost':[cost_a],
        'produksi':[produksi_a], 
        'income':[income_a],
        'profit':[profit_a],
        'pm':[pm_a]
    })

    msg['condition'] = name
    _.append(msg)

report = pd.concat(_)[['condition','cost','produksi','income','profit','pm']]
report = report[report.condition!='0HY + 0ZY']
report = pd.concat([report, a])

report.sort_values('pm', ascending=False).reset_index(drop=True).to_excel('Kondisi jika menambah DT pada lokasi KBM namun hanya bisa mendapatkan target produksi 110,151 Ton.xlsx', index=False)
report  = report.sort_values('pm', ascending=False).reset_index(drop=True)
report

,condition,cost,produksi,income,profit,pm
0,0HY + 5ZY,1.240758e+10,110151.00,1.717298e+10,4.765404e+09,0.277494
1,1HY + 4ZY,1.245091e+10,110151.00,1.717298e+10,4.722076e+09,0.274971
2,2HY + 3ZY,1.249423e+10,110151.00,1.717298e+10,4.678748e+09,0.272448
3,3HY + 2ZY,1.253756e+10,110151.00,1.717298e+10,4.635420e+09,0.269925
4,4HY + 1ZY,1.258089e+10,110151.00,1.717298e+10,4.592093e+09,0.267402
5,5HY + 0ZY,1.262422e+10,110151.00,1.717298e+10,4.548765e+09,0.264879
6,0HY + 0ZY,1.182016e+10,94399.75,1.471730e+10,2.897143e+09,0.196853


## Task 2 :

## Task 3 :

## Task 4 :

## TASK 5 : DASHBOARD COST FUEL EFFICIENCY

In [196]:
# prepare date kpi
task6 = kpi[['date','company','project/name','production','move_type','category_equipment',
             'equipment_name','name','jam_produktivitas','retase','total_jarak_tempuh','produksi']].rename(columns={'jam_produktivitas':'hm','total_jarak_tempuh':'km'})
task6['id'] = task6['date'] + '#' + task6['name']

# prepare data ooc
task6_ = ooc.copy()
task6_['id'] = task6_['date'] + '#' + task6_['name']

# merge data kpi dan ooc
task6 = task6.merge(task6_[['id','rm','fuel','tyre']], on='id', how='left')

# merge data dengan actual operasional per hm
task6 = task6.merge(cc5[['equipment_name','rm/hm','fuel/hm','tyre/hm']], on='equipment_name', how='left')
task6['max_rm'] = task6.hm * task6['rm/hm']
task6['max_fuel'] = task6.hm * task6['fuel/hm']
task6['max_tyre'] = task6.hm * task6['tyre/hm']

# re-columns task6
task6 = task6[['date','company','project/name','production','move_type','category_equipment','equipment_name','name','hm','rm','fuel','tyre','retase','km','produksi','max_rm','max_fuel','max_tyre']]
task6 = task6.drop_duplicates(subset=['date','project/name','production','move_type','name'])
task6.to_excel('./Report/fuel efficiency.xlsx', index=False)

In [197]:
task6_1 = task6.groupby(['date','project/name'])[['max_fuel','fuel']].sum().reset_index()
task6_1 = task6_1[task6_1['project/name'].isin(['Mining BDM','Mining KBM','Rental BDM'])]

def in_out_control(max_fuel, fuel):
    if (max_fuel - fuel) > 0:
        msg = 'in Control'
    else:
        msg = 'out Control'
    return msg

task6_1['status'] = task6_1.apply(lambda x: in_out_control(x.max_fuel, x.fuel), axis=1)
task6_1.to_excel('./Report/fuel efficiency 2.xlsx', index=False)

## TASK 6 :

## TASK 7 : CLEANING AND TRANSFORMING DATA TASK 2 : BEP SA & APP

In [198]:
c = pd.read_excel('Rekap Ban , Oli dan Grease SA APP 2019-2023.xlsx', sheet_name='REKAP GLOBAL')
c = c[2:]
c = c[['TAHUN ','Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12',
       'Unnamed: 13','Unnamed: 14','Unnamed: 15','Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19','Unnamed: 20','Unnamed: 21','Unnamed: 22','Unnamed: 23','Unnamed: 24',
       'Unnamed: 25','Unnamed: 26','Unnamed: 27','Unnamed: 28','Unnamed: 29','Unnamed: 30','Unnamed: 31','Unnamed: 32','Unnamed: 33','Unnamed: 34','Unnamed: 35','Unnamed: 36']] 

# create column names
cols = ['no','name','company','tyre_22','tyre_21','tyre_20','tyre_19','oil_23','grase_23','priceoil_23','pricegrease_23','totaloil_23','totalgrease_23'
        ,'oil_22','grease_22','priceoil_22','pricegrease_22','totaloil_22','totalgrease_22',
        'oil_21','grease_21','priceoil_21','pricegrease_21','totaloil_21','totalgrease_21',
        'oil_20','grease_20','priceoil_20','pricegrease_20','totaloil_20','totalgrease_20',
        'oil_19','grease_19','priceoil_19','pricegrease_19','totaloil_19','totalgrease_19']

c.columns = cols
drop_cols = ['tyre_22','tyre_21','tyre_20','tyre_19']
c = c.drop(columns=drop_cols)
c = c[c.name!='Total']

# data transforming
_ = []
for i in ['oil','grease']:
    if i == 'oil':
        catMain = 'OLI'
    else:
        catMain = 'GREASE'

    for j in np.arange(19, 22):
        price = 'price' + i + '_' + str(j)
        qty = i + '_' + str(j)
        total = 'total' + i + '_' + str(j)
        year = str(j)

        data = c[['name','company',qty,price,total]]
        data['year'] = int(year)
        data['category_maintenance'] = catMain
        data['year'] = int('20' + str(j))
        data.columns = ['name','company','qty','price','cost','year','category_maintenance']
        _.append(data)
data1 = pd.concat(_)[['year','company','name','category_maintenance','qty','price','cost']]
data1['name'] = data1.name.apply(lambda x: '-'.join(str(x).split()))

# read data tyre
b = pd.read_excel('Rekap Ban , Oli dan Grease SA APP 2019-2023.xlsx', sheet_name='REKAP GLOBAL BAN')
b = b[2:]
b = b[:-1]
b.columns = ['name','company','qty_19','price_19','cost_19','qty_20','price_20','cost_20','qty_21','price_21','cost_21','qty_22','price_22','cost_22','qty_23','price_23','cost_23']

# transformasi data tyre
_ = []
for i in np.arange(19, 23):
    price = 'price_' + str(i)
    qty = 'qty_' + str(i)
    cost = 'cost_' + str(i)

    data = b[['name','company',qty,price,cost]]
    data['category_maintenance'] = 'TYRE'
    data['year'] = int('20' + str(i))
    data.columns = ['name','company','qty','price','cost','category_maintenance','year']
    data = data[['year','company','name','category_maintenance','qty','price','cost']]
    _.append(data)
    
data2 = pd.concat(_)
data2['name'] = data2['name'].apply(lambda x: '-'.join(str(x).split()))

# gabung data1 dan data2
data = pd.concat([data1, data2])
data.fillna(0, inplace=True)
data = data[data.year <= 2023]

In [199]:
rupiah(data.groupby(['name','category_maintenance'])[['cost']].sum().sum()['cost']).split(',')[0]

'Rp 55.623.282.923'

In [200]:
n = data.groupby(['category_maintenance'])[['cost']].sum()
n['cost'] = n['cost'].apply(lambda x: str(rupiah(x)).split(',')[0])
n

,cost
category_maintenance,
GREASE,Rp 308.504.100
OLI,Rp 107.956.041
TYRE,Rp 55.206.822.782
